<center>
<img src="../../img/ods_stickers.jpg">
## Открытый курс по машинному обучению. Сессия № 2
</center>
Автор материала: программист-исследователь Mail.ru Group, старший преподаватель Факультета Компьютерных Наук ВШЭ Юрий Кашницкий. Материал распространяется на условиях лицензии [Creative Commons CC BY-NC-SA 4.0](https://creativecommons.org/licenses/by-nc-sa/4.0/). Можно использовать в любых целях (редактировать, поправлять и брать за основу), кроме коммерческих, но с обязательным упоминанием автора материала.

# <center>Домашнее задание № 8
## <center> Vowpal Wabbit в задаче прогнозирования популярности статьи на хабре

В этом задании надо побить бенчмарк в [соревновании](https://www.kaggle.com/c/habr-num-bookmarks) на Kaggle Inclass. Как это делать – ограничений нет (кроме, конечно, ручной разметки), прочитать правила можно [тут](https://www.kaggle.com/c/habr-num-bookmarks/rules). Ниже описаны инструкции, как это сделать с Vowpal Wabbit.

Дедлайн: 31 октября 23:59 UTC +3. Решение надо будет загрузить по [ссылке](https://www.dropbox.com/request/g5WOPrxwvcYwADZCuoY7). В этом соревновании нет задачи победить. Цель – побить бенчмарк и продвинуться в [соревновании](https://mlcourse.arktur.io) по прогнозу популярности статьи на Medium. 

In [2]:
import numpy as np
import pandas as pd
import json
from tqdm import tqdm_notebook
from sklearn.metrics import mean_absolute_error
import datetime

Посмотрим на одну из строчек в JSON-файле: считаем ее с помощью библиотеки json. Эта строчка соответствует [7-ой статье](https://habrahabr.ru/post/7/) на Хабре.

In [1]:
!head -1 D:/train.json > D:/train1.json

"head" ­Ґ пў«пҐвбп ў­гваҐ­­Ґ© Ё«Ё ў­Ґи­Ґ©
Є®¬ ­¤®©, ЁбЇ®«­пҐ¬®© Їа®Ја ¬¬®© Ё«Ё Ї ЄҐв­л¬ д ©«®¬.


In [5]:
with open('D:\\train1.json') as inp_json:
    first_json = json.load(inp_json)

In [6]:
first_json.keys()

dict_keys(['author', 'content', 'domain', 'meta_tags', 'tags', 'flags', 'link_tags', 'polling', 'flow', 'post_id', 'hubs', 'url', '_id', '_timestamp', 'published', 'title'])

Видим 16 полей, перечислим некоторые из них:
- _id, url - URL статьи
- published – время публикации статьи
- domain – сайт (например, habrahahbr.ru или geektimes.ru)
- title – название статьи
- content – текст статьи
- hubs - перечисление хабов, к которым относится статья
- tags – теги статьи
- author – автор статьи, его ник и ссылка на профиль

In [7]:
first_json['_id']

'https://habrahabr.ru/post/7/'

In [8]:
first_json['_timestamp']

1493192186.0903192

In [9]:
first_json['url']

'https://habrahabr.ru/post/7/'

In [10]:
first_json['domain']

'habrahabr.ru'

In [9]:
first_json['published']

{'$date': '2006-07-15T01:48:00.000Z'}

In [10]:
first_json['title']

'Самопроизвольное разлогинивание'

In [11]:
first_json['content']

'У меня такое ощущение, что logout время от времени происходит самопроизвольно, несмотря на то, что чекбокс про логине включен.<br>\r\n<br>\r\nВозможно, это происходит при смене IP-адреса, но я не уверен.'

In [12]:
first_json['polling']

In [13]:
first_json['post_id']

7

In [14]:
first_json['flags']

[]

In [15]:
first_json['hubs']

[{'id': 'hub/habr',
  'title': 'Хабрахабр',
  'url': 'https://habrahabr.ru/hub/habr/'}]

In [16]:
first_json['flow']

In [17]:
first_json['tags']

['логин', 'login']

In [18]:
first_json['author']

{'name': 'Павел Титов',
 'nickname': '@ptitov',
 'url': 'https://habrahabr.ru/users/ptitov'}

In [19]:
first_json['link_tags']

{'alternate': 'https://habrahabr.ru/rss/post/7/',
 'apple-touch-icon-precomposed': '/images/favicons/apple-touch-icon-152x152.png',
 'canonical': 'https://habrahabr.ru/post/7/',
 'icon': '/images/favicons/favicon-16x16.png',
 'image_src': 'https://habrahabr.ru/i/habralogo.jpg',
 'stylesheet': 'https://habracdn.net/habr/styles/1493134745/_build/global_main.css'}

In [20]:
first_json['meta_tags']

{'al:android:app_name': 'Habrahabr',
 'al:android:package': 'ru.habrahabr',
 'al:android:url': 'habrahabr://post/7',
 'al:windows_phone:app_id': '460a6bd6-8955-470f-935e-9ea1726a6060',
 'al:windows_phone:app_name': 'Habrahabr',
 'al:windows_phone:url': 'habrahabr://post/7',
 'apple-mobile-web-app-title': 'Хабрахабр',
 'application-name': 'Хабрахабр',
 'description': 'У меня такое ощущение, что logout время от времени происходит самопроизвольно, несмотря на то, что чекбокс про логине включен.\r\n\r\nВозможно, это происходит при смене IP-адреса, но я не уверен.',
 'fb:app_id': '444736788986613',
 'keywords': 'логин, login',
 'msapplication-TileColor': '#FFFFFF',
 'msapplication-TileImage': 'mstile-144x144.png',
 'og:description': 'У меня такое ощущение, что logout время от времени происходит самопроизвольно, несмотря на то, что чекбокс про логине включен.  Возможно, это происходит при...',
 'og:image': 'https://habrahabr.ru/i/habralogo.jpg',
 'og:title': 'Самопроизвольное разлогинивание'

Загрузим ответы на обучающей выборке.

In [3]:
train_target = pd.read_csv('D:/train_target.csv',
                          index_col='url')

In [22]:
train_target.head()

,target
url,
https://habrahabr.ru/post/7/,0.693147
https://geektimes.ru/post/11/,1.098612
https://geektimes.ru/post/112/,0.000000
https://geektimes.ru/post/1127/,0.000000
https://geektimes.ru/post/12664/,0.000000


In [40]:
train_target[train_target.index == train_target.index[0]]

,target
url,
https://habrahabr.ru/post/7/,0.693147


In [4]:
import re 
def ModifyStr(astr):
    return(re.sub(r'([^\s\w]|_)+', '', astr.lower()))
    #return(re.sub(r'([^\s\w]|_)+', '', astr))

    
def TagsToStr(tags):
    if len(tags) == 0:
        return(' None')
    else:
        astr = ''
        for j in range(len(tags)):
            astr += ' '  + ModifyStr(str(tags[j]))
        return(astr)
    
def HubsToStr(hubs):
    if len(hubs) == 0:
        return(' None')
    else:
        astr = ''
        for j in range(len(hubs)):
            astr += ' ' + ModifyStr(str(hubs[j]['title']))
        return(astr)

In [154]:
count = 0
with open('D:/train.json', 'r') as f:
    for line in f:
        traindat = json.loads(line)
        url = traindat['url']
        prob = train_target[train_target.index == url].values[0][0]
        #print(url, prob, traindat['title'], traindat['tags'], traindat['domain'], traindat['flow'], traindat['author'], traindat['hubs'])
        #print(traindat)
        astr = str(prob) + ' |'
        #atitle = traindat['title'].strip('!?,.-+=*::|/\\').strip(':').lower()
        atitle = ModifyStr(traindat['title'])
        #print(atitle.strip('='))
        astr += 'title ' + str(atitle) + ' |'
        astr += 'tags' + TagsToStr(traindat['tags']) + ' |'
        astr += 'domain ' + ModifyStr(str(traindat['domain'])) + ' |'
        astr += 'flow ' + ModifyStr(str(traindat['flow'])) + ' |'
        astr += 'author ' + ModifyStr(str(traindat['author']['nickname'])) + ' |'
        astr += 'hubs ' + HubsToStr(traindat['hubs']) + ' |'
        astr += 'num content_len:' + str(len(traindat['content'])/1000000)
        atime = datetime.datetime.strptime(traindat['published']['$date'][0:10] + ' ' + traindat['published']['$date'][11:19], '%Y-%m-%d %H:%M:%S')
        #print(atime)
        astr += ' meanword:' + str(traindat['content'].count(' ')/(0.0001+len(traindat['content'])))
        astr += ' dayofweek:' + str(atime.weekday())
        #astr += ' month:' + str(atime.month)
        astr += '\n'
        #astr += ' hour:' + str(atime.hour) + '\n'
        #print(astr)
        with open('D:/habr_train_alt4.vw', 'a', encoding = 'utf-8') as g:
            g.write(astr)
        g.close()
        count += 1
        #if count > 10:
        #    break
        if count % 1000 == 0:
            print(count)
f.close()

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000


In [149]:
traindat['author']

{'name': 'ayambit',
 'nickname': '@ayambit',
 'url': 'https://geektimes.ru/users/ayambit'}

In [113]:
datetime.datetime.strptime(traindat['published']['$date'][0:10] + ' ' + traindat['published']['$date'][11:19], '%Y-%m-%d %H:%M:%S')
#datetime.datetime.strptime(traindat['published']['$date'], '%Y-%m-%d')

datetime.datetime(2006, 7, 15, 1, 48)

Сформируйте обучающую выборку для Vowpal Wabbit, выберите признаки title, title, tags, domain, flow, author, и hubs из JSON-файла.
От самого текста для начала просто возьмем его длину – постройте признак content_len – длина текста в миллионнах символов.
Также постройте признаки – час и месяц публикации статьи. Еще, конечно же, возьмите ответы на обучающей выборке из `train_target`. Ниже пример того, как могут выглядеть первые две строки нового файла.

In [23]:
!head -2 ../../data/habr_train.vw

0.6931470000000001 |title Самопроизвольное разлогинивание |tags логин login |domain habrahabr.ru |flow None |author @ptitov |hubs Хабрахабр |num content_len:0.0 month:7 hour:1
1.0986120000000001 |title Stand-along cообщества против сообществ в рамках социальных сетей |tags сообщества интернет-сообщество социальные сети нишевой бренд |domain geektimes.ru |flow None |author @AlexBruce |hubs Чёрная дыра |num content_len:0.0 month:7 hour:14


In [ ]:
with open('../../data/train.json') as inp_json, \
     open('../../data/habr_train.vw', 'w') as out_vw:
    for line in tqdm_notebook(inp_json):
        data_json = json.loads(line)
        
        # Ваш код здесь

Проделайте все то же с тестовой выборкой, вместо ответов подсовывая что угодно, например, единицы.

In [124]:
count = 0
with open('D:/test.json', 'r') as f:
    for line in f:
        traindat = json.loads(line)
        url = traindat['url']
        prob = 1
        #print(url, prob, traindat['title'], traindat['tags'], traindat['domain'], traindat['flow'], traindat['author'], traindat['hubs'])
        #print(traindat)
        astr = str(prob) + ' |'
        astr += 'title ' + str(traindat['title']) + ' |'
        astr += 'tags' + TagsToStr(traindat['tags']) + ' |'
        astr += 'domain ' + str(traindat['domain']) + ' |'
        astr += 'flow ' + str(traindat['flow']) + ' |'
        astr += 'author ' + str(traindat['author']['name']) + ' |'
        astr += 'hubs ' + HubsToStr(traindat['hubs']) + ' |'
        astr += 'num content_len:' + str(len(traindat['content'])/1000000)
        atime = datetime.datetime.strptime(traindat['published']['$date'][0:10] + ' ' + traindat['published']['$date'][11:19], '%Y-%m-%d %H:%M:%S')
        #print(atime)
        astr += ' month:' + str(atime.month)
        astr += ' hour:' + str(atime.hour) + '\n'
        #print(astr)
        with open('D:/habr_test.vw', 'a', encoding = 'utf-8') as g:
            g.write(astr)
        g.close()
        count += 1
        if count % 1000 == 0:
            print(count)
f.close()

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000


In [24]:
!head -2 ../../data/habr_test.vw

1 |title День Пи! |tags Пи Pi |domain geektimes.ru |flow None |author @Timursan |hubs Чёрная дыра |num content_len:0.0 month:3 hour:3
1 |title Скрипт для разбиения образов музыкальных CD на треки и конвертации в формат FLAC |tags bash lossless |domain geektimes.ru |flow None |author @da3mon |hubs Чёрная дыра |num content_len:0.01 month:3 hour:0


Выбор того, как валидировать модель, остается за Вами. Проще всего, конечно, сделать отложенную выборку. Бенчмарк, который Вы видите в соревновании (**vw_baseline.csv**) и который надо побить, получен с Vowpal Wabbit, 3 проходами по выборке (не забываем удалять кэш), биграммами и настроенными гиперпараметрами `bits`, `learning_rate` и `power_t`. 

In [155]:
with open('D:/habr_train_alt4_pred1.txt') as pred_file:
    valid_prediction = [float(label) 
                             for label in pred_file.readlines()]

In [227]:
with open('D:/habr_train_alt4_model1.txt') as pred_file:
    valid_prediction = [float(label) 
                             for label in pred_file.readlines()]

In [228]:
sklearn.metrics.mean_absolute_error(valid_prediction, train_target.values)

0.75194658837499995

In [229]:
with open('D:/habr_train_alt4_model2.txt') as pred_file:
    valid_prediction = [float(label) 
                             for label in pred_file.readlines()]

In [230]:
sklearn.metrics.mean_absolute_error(valid_prediction, train_target.values)

0.75590993850833343

In [158]:
len(valid_prediction), len(train_target.values)

(120000, 120000)

In [164]:
import sklearn.metrics
sklearn.metrics.mean_absolute_error(valid_prediction, train_target.values), sklearn.metrics.mean_squared_error(valid_prediction, train_target.values)

(0.75590993850833343, 0.97563168241151366)

In [166]:
with open('D:/habr_train_alt2_pred1.txt') as pred_file:
    valid_prediction = [float(label) 
                             for label in pred_file.readlines()]

In [167]:
sklearn.metrics.mean_absolute_error(valid_prediction, train_target.values), sklearn.metrics.mean_squared_error(valid_prediction, train_target.values)

(0.75325537103333329, 0.95012043376012634)

In [38]:
import random

In [39]:
myindices = range(0,  120001)

In [40]:
random.seed(239)
myindices1 = random.sample(myindices, 84000)

In [41]:
myindices1[0:10]

[15560, 25104, 108474, 93136, 19637, 54647, 13521, 57160, 8087, 48207]

In [198]:
count = 0
with open('D:/train.json', 'r') as f:
    for line in f:
        traindat = json.loads(line)
        url = traindat['url']
        prob = train_target[train_target.index == url].values[0][0]
        #print(url, prob, traindat['title'], traindat['tags'], traindat['domain'], traindat['flow'], traindat['author'], traindat['hubs'])
        #print(traindat)
        astr = str(prob) + ' |'
        #atitle = traindat['title'].strip('!?,.-+=*::|/\\').strip(':').lower()
        atitle = ModifyStr(traindat['title'])
        #print(atitle.strip('='))
        astr += 'title ' + str(atitle) + ' |'
        astr += 'tags' + TagsToStr(traindat['tags']) + ' |'
        astr += 'domain ' + ModifyStr(str(traindat['domain'])) + ' |'
        astr += 'flow ' + ModifyStr(str(traindat['flow'])) + ' |'
        astr += 'author ' + ModifyStr(str(traindat['author']['nickname'])) + ' |'
        astr += 'hubs ' + HubsToStr(traindat['hubs']) + ' |'
        astr += 'num content_len:' + str(len(traindat['content'])/1000000)
        atime = datetime.datetime.strptime(traindat['published']['$date'][0:10] + ' ' + traindat['published']['$date'][11:19], '%Y-%m-%d %H:%M:%S')
        #print(atime)
        #astr += ' meanword:' + str(traindat['content'].count(' ')/(0.0001+len(traindat['content'])))
        #astr += ' dayofweek:' + str(atime.weekday())
        astr += ' month:' + str(atime.month)
        #astr += '\n'
        astr += ' hour:' + str(atime.hour) + '\n'
        #print(astr)
        if count in myindices1:
            with open('D:/habr_minitrain_alt2.vw', 'a', encoding = 'utf-8') as g:
                g.write(astr)
        else:
             with open('D:/habr_minitest_alt2.vw', 'a', encoding = 'utf-8') as g:
                g.write(astr)           
        g.close()
        count += 1
        #if count > 10:
        #    break
        if count % 1000 == 0:
            print(count)
f.close()

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000


In [45]:
count = 0
with open('D:/train.json', 'r') as f:
    for line in f:
        traindat = json.loads(line)
        url = traindat['url']
        prob = train_target[train_target.index == url].values[0][0]
        #print(url, prob, traindat['title'], traindat['tags'], traindat['domain'], traindat['flow'], traindat['author'], traindat['hubs'])
        #print(traindat)
        astr = str(prob) + ' |'
        #atitle = traindat['title'].strip('!?,.-+=*::|/\\').strip(':').lower()
        atitle = ModifyStr(traindat['title'])
        #print(atitle.strip('='))
        astr += 'title ' + str(atitle) + ' |'
        astr += 'tags' + TagsToStr(traindat['tags']) + ' |'
        astr += 'domain ' + ModifyStr(str(traindat['domain'])) + ' |'
        astr += 'flow ' + ModifyStr(str(traindat['flow'])) + ' |'
        astr += 'author ' + ModifyStr(str(traindat['author']['nickname'])) + ' |'
        astr += 'content ' + ModifyStr(traindat['content']).replace('\n', ' ').replace('\t', ' ') + ' |'
        astr += 'hubs ' + HubsToStr(traindat['hubs']) + ' |'
        astr += 'num content_len:' + str(len(traindat['content'])/1000000)
        atime = datetime.datetime.strptime(traindat['published']['$date'][0:10] + ' ' + traindat['published']['$date'][11:19], '%Y-%m-%d %H:%M:%S')
        #print(atime)
        astr += ' meanword:' + str(round(traindat['content'].count(' ')/(0.0001+len(traindat['content'])), 3))
        astr += ' dayofweek:' + str(atime.weekday())
        astr += ' month:' + str(atime.month)
        #astr += '\n'
        astr += ' hour:' + str(atime.hour) + '\n'
        #print(astr)
        if count in myindices1:
            with open('D:/habr_minitrain_alt0f.vw', 'a', encoding = 'utf-8') as g:
                g.write(astr)
        else:
             with open('D:/habr_minitest_alt0f.vw', 'a', encoding = 'utf-8') as g:
                g.write(astr)           
        g.close()
        with open('D:/habr_train_alt0f.vw', 'a', encoding = 'utf-8') as g:
            g.write(astr)
        g.close()
        count += 1
        #if count > 10:
        #    break
        if count % 1000 == 0:
            print(count)
f.close()

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000


In [5]:
count = 0
traintargetan = []
testtargetan = []
with open('D:/train.json', 'r') as f:
    for line in f:
        traindat = json.loads(line)
        url = traindat['url']
        prob = train_target[train_target.index == url].values[0][0]
        #print(url, prob, traindat['title'], traindat['tags'], traindat['domain'], traindat['flow'], traindat['author'], traindat['hubs'])
        #print(traindat)
        astr = str(prob) + ' |'
        #atitle = traindat['title'].strip('!?,.-+=*::|/\\').strip(':').lower()
        atitle = ModifyStr(traindat['title'])
        #print(atitle.strip('='))
        astr += 'title ' + str(atitle) + ' |'
        astr += 'tags' + TagsToStr(traindat['tags']) + ' |'
        astr += 'domain ' + ModifyStr(str(traindat['domain'])) + ' |'
        astr += 'flow ' + ModifyStr(str(traindat['flow'])) + ' |'
        astr += 'author ' + ModifyStr(str(traindat['author']['nickname'])) + ' |'
        astr += 'hubs ' + HubsToStr(traindat['hubs']) + ' |'
        astr += 'num content_len:' + str(len(traindat['content'])/1000000)
        atime = datetime.datetime.strptime(traindat['published']['$date'][0:10] + ' ' + traindat['published']['$date'][11:19], '%Y-%m-%d %H:%M:%S')
        #print(atime)
        astr += ' meanword:' + str(round(traindat['content'].count(' ')/(0.0001+len(traindat['content'])), 3))
        astr += ' dayofweek:' + str(atime.weekday())
        astr += ' month:' + str(atime.month)
        #astr += '\n'
        astr += ' hour:' + str(atime.hour) + '\n'
        #print(astr)
        if count < 84000:
            #with open('D:/habr_minitrain_alt0_an.vw', 'a', encoding = 'utf-8') as g:
            #    g.write(astr)
            traintargetan.append(prob)
        else:
            #with open('D:/habr_minitest_alt0_an.vw', 'a', encoding = 'utf-8') as g:
            #    g.write(astr)
            testtargetan.append(prob)
        #g.close()
        count += 1
        #if count > 10:
        #    break
        if count % 1000 == 0:
            print(count)
f.close()

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000


In [95]:
count = 0
with open('D:/train.json', 'r') as f:
    for line in f:
        traindat = json.loads(line)
        url = traindat['url']
        prob = train_target[train_target.index == url].values[0][0]
        #print(url, prob, traindat['title'], traindat['tags'], traindat['domain'], traindat['flow'], traindat['author'], traindat['hubs'])
        #print(traindat)
        astr = str(prob) + ' |'
        #atitle = traindat['title'].strip('!?,.-+=*::|/\\').strip(':').lower()
        atitle = ModifyStr(traindat['title'])
        #print(atitle.strip('='))
        astr += 'title ' + str(atitle) + ' |'
        astr += 'tags' + TagsToStr(traindat['tags']) + ' |'
        astr += 'domain ' + ModifyStr(str(traindat['domain'])) + ' |'
        astr += 'flow ' + ModifyStr(str(traindat['flow'])) + ' |'
        astr += 'author ' + ModifyStr(str(traindat['author']['nickname'])) + ' |'
        astr += 'hubs ' + HubsToStr(traindat['hubs']) + ' |'
        astr += 'num content_len:' + str(round(len(traindat['content'])/1000000, 2))
        atime = datetime.datetime.strptime(traindat['published']['$date'][0:10] + ' ' + traindat['published']['$date'][11:19], '%Y-%m-%d %H:%M:%S')
        #print(atime)
        astr += ' meanword:' + str(round(traindat['content'].count(' ')/(0.0001+len(traindat['content'])), 2))
        #astr += ' numspaces:' + str(traindat['content'].count(' ')/10000)
        astr += ' dayofweek:' + str(atime.weekday())
        astr += ' month:' + str(atime.month)
        #astr += '\n'
        astr += ' hour:' + str(atime.hour) + '\n'
        #print(astr)
        if count in myindices1:
            with open('D:/habr_minitrain_alt0final.vw', 'a', encoding = 'utf-8') as g:
                g.write(astr)
        else:
             with open('D:/habr_minitest_alt0final.vw', 'a', encoding = 'utf-8') as g:
                g.write(astr)           
        g.close()
        with open('D:/habr_train_alt0final.vw', 'a', encoding = 'utf-8') as g:
            g.write(astr)
        g.close()
        count += 1
        #if count > 10:
        #    break
        if count % 1000 == 0:
            print(count)
f.close()

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000


In [153]:
count = 0
with open('D:/test.json', 'r') as f:
    for line in f:
        traindat = json.loads(line)
        url = traindat['url']
        #prob = train_target[train_target.index == url].values[0][0]
        #print(url, prob, traindat['title'], traindat['tags'], traindat['domain'], traindat['flow'], traindat['author'], traindat['hubs'])
        #print(traindat)
        astr = str(0) + ' |'
        #atitle = traindat['title'].strip('!?,.-+=*::|/\\').strip(':').lower()
        atitle = ModifyStr(traindat['title'])
        #print(atitle.strip('='))
        astr += 'title ' + str(atitle) + ' |'
        astr += 'tags' + TagsToStr(traindat['tags']) + ' |'
        astr += 'domain ' + ModifyStr(str(traindat['domain'])) + ' |'
        astr += 'flow ' + ModifyStr(str(traindat['flow'])) + ' |'
        astr += 'author ' + ModifyStr(str(traindat['author']['nickname'])) + ' |'
        astr += 'hubs ' + HubsToStr(traindat['hubs']) + ' |'
        astr += 'num content_len:' + str(round(len(traindat['content'])/1000000, 2))
        atime = datetime.datetime.strptime(traindat['published']['$date'][0:10] + ' ' + traindat['published']['$date'][11:19], '%Y-%m-%d %H:%M:%S')
        #print(atime)
        astr += ' meanword:' + str(round(traindat['content'].count(' ')/(0.0001+len(traindat['content'])), 2))
        #astr += ' numspaces:' + str(traindat['content'].count(' ')/10000)
        astr += ' dayofweek:' + str(atime.weekday())
        astr += ' month:' + str(atime.month)
        #astr += '\n'
        astr += ' hour:' + str(atime.hour) + '\n'
        #print(astr)
        #if count in myindices1:
        #    with open('D:/habr_minitrain_alt0final.vw', 'a', encoding = 'utf-8') as g:
        #        g.write(astr)
        #else:
        #     with open('D:/habr_minitest_alt0final.vw', 'a', encoding = 'utf-8') as g:
        #        g.write(astr)           
        #g.close()
        with open('D:/habr_test_alt0final.vw', 'a', encoding = 'utf-8') as g:
            g.write(astr)
        g.close()
        count += 1
        #if count > 10:
        #    break
        if count % 1000 == 0:
            print(count)
f.close()

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000


In [540]:
count = 0
with open('D:/test.json', 'r') as f:
    for line in f:
        traindat = json.loads(line)
        url = traindat['url']
        #prob = train_target[train_target.index == url].values[0][0]
        #print(url, prob, traindat['title'], traindat['tags'], traindat['domain'], traindat['flow'], traindat['author'], traindat['hubs'])
        #print(traindat)
        astr = str(1) + ' |'
        #atitle = traindat['title'].strip('!?,.-+=*::|/\\').strip(':').lower()
        atitle = ModifyStr(traindat['title'])
        #print(atitle.strip('='))
        astr += 'title ' + str(atitle) + ' |'
        astr += 'tags' + TagsToStr(traindat['tags']) + ' |'
        astr += 'domain ' + ModifyStr(str(traindat['domain'])) + ' |'
        astr += 'flow ' + ModifyStr(str(traindat['flow'])) + ' |'
        astr += 'author ' + ModifyStr(str(traindat['author']['nickname'])) + ' |'
        astr += 'hubs ' + HubsToStr(traindat['hubs']) + ' |'
        astr += 'num content_len:' + str(len(traindat['content'])/1000000)
        atime = datetime.datetime.strptime(traindat['published']['$date'][0:10] + ' ' + traindat['published']['$date'][11:19], '%Y-%m-%d %H:%M:%S')
        #print(atime)
        astr += ' meanword:' + str(round(traindat['content'].count(' ')/(0.0001+len(traindat['content'])), 3))
        astr += ' dayofweek:' + str(atime.weekday())
        astr += ' month:' + str(atime.month)
        #astr += '\n'
        astr += ' hour:' + str(atime.hour) + '\n'
        #print(astr)
        #if count in myindices1:
        #    with open('D:/habr_minitrain_alt0.vw', 'a', encoding = 'utf-8') as g:
        #        g.write(astr)
        #else:
        #     with open('D:/habr_minitest_alt0.vw', 'a', encoding = 'utf-8') as g:
        #        g.write(astr)           
        #g.close()
        with open('D:/habr_test_alt0.vw', 'a', encoding = 'utf-8') as g:
            g.write(astr)
        g.close()
        count += 1
        #if count > 10:
        #    break
        if count % 1000 == 0:
            print(count)
f.close()

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000


In [500]:
count = 0
with open('D:/train.json', 'r') as f:
    for line in f:
        traindat = json.loads(line)
        url = traindat['url']
        prob = train_target[train_target.index == url].values[0][0]
        #print(url, prob, traindat['title'], traindat['tags'], traindat['domain'], traindat['flow'], traindat['author'], traindat['hubs'])
        #print(traindat)
        astr = str(prob) + ' |'
        #atitle = traindat['title'].strip('!?,.-+=*::|/\\').strip(':').lower()
        atitle = ModifyStr(traindat['title'])
        #print(atitle.strip('='))
        astr += 'title ' + str(atitle) + ' |'
        astr += 'tags' + TagsToStr(traindat['tags']) + ' |'
        astr += 'domain ' + ModifyStr(str(traindat['domain'])) + ' |'
        astr += 'flow ' + ModifyStr(str(traindat['flow'])) + ' |'
        astr += 'author ' + ModifyStr(str(traindat['author']['nickname'])) + ' |'
        astr += 'hubs ' + HubsToStr(traindat['hubs']) + ' |'
        astr += 'num content_len:' + str(len(traindat['content'])/1000000)
        atime = datetime.datetime.strptime(traindat['published']['$date'][0:10] + ' ' + traindat['published']['$date'][11:19], '%Y-%m-%d %H:%M:%S')
        #print(atime)
        #astr += ' meanword:' + str(traindat['content'].count(' ')/(0.0001+len(traindat['content'])))
        #astr += ' dayofweek:' + str(atime.weekday())
        astr += ' month:' + str(atime.month)
        #astr += '\n'
        astr += ' hour:' + str(atime.hour) + '\n'
        #print(astr)
        if count < 84000:
            with open('D:/habr_minitrain_altnew.vw', 'a', encoding = 'utf-8') as g:
                g.write(astr)
        else:
             with open('D:/habr_minitest_altnew.vw', 'a', encoding = 'utf-8') as g:
                g.write(astr)           
        g.close()
        count += 1
        #if count > 10:
        #    break
        if count % 1000 == 0:
            print(count)
f.close()

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000


In [282]:
count = 0
with open('D:/test.json', 'r') as f:
    for line in f:
        traindat = json.loads(line)
        url = traindat['url']
        #prob = train_target[train_target.index == url].values[0][0]
        #print(url, prob, traindat['title'], traindat['tags'], traindat['domain'], traindat['flow'], traindat['author'], traindat['hubs'])
        #print(traindat)
        astr = str(1) + ' |'
        #atitle = traindat['title'].strip('!?,.-+=*::|/\\').strip(':').lower()
        atitle = ModifyStr(traindat['title'])
        #print(atitle.strip('='))
        astr += 'title ' + str(atitle) + ' |'
        astr += 'tags' + TagsToStr(traindat['tags']) + ' |'
        astr += 'domain ' + ModifyStr(str(traindat['domain'])) + ' |'
        astr += 'flow ' + ModifyStr(str(traindat['flow'])) + ' |'
        astr += 'author ' + ModifyStr(str(traindat['author']['nickname'])) + ' |'
        astr += 'hubs ' + HubsToStr(traindat['hubs']) + ' |'
        astr += 'num content_len:' + str(len(traindat['content'])/1000000)
        atime = datetime.datetime.strptime(traindat['published']['$date'][0:10] + ' ' + traindat['published']['$date'][11:19], '%Y-%m-%d %H:%M:%S')
        #print(atime)
        #astr += ' meanword:' + str(traindat['content'].count(' ')/(0.0001+len(traindat['content'])))
        #astr += ' dayofweek:' + str(atime.weekday())
        astr += ' month:' + str(atime.month)
        #astr += '\n'
        astr += ' hour:' + str(atime.hour) + '\n'
        #print(astr)
        #if count in myindices1:
        #    with open('D:/habr_minitrain_alt2.vw', 'a', encoding = 'utf-8') as g:
        #        g.write(astr)
        #else:
        with open('D:/habr_test_alt2.vw', 'a', encoding = 'utf-8') as g:
            g.write(astr)           
        g.close()
        count += 1
        #if count > 10:
        #    break
        if count % 1000 == 0:
            print(count)
f.close()

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000


In [22]:
sample_sub = pd.read_csv('D:/sample_submission.csv', index_col='url')

In [235]:
with open('D:/habr_minitrain_alt4_model2.txt') as pred_file:
    valid_predictiontrain = [float(label) 
                             for label in pred_file.readlines()]
with open('D:/habr_minitest_alt4_model2.txt') as pred_file:
    valid_predictiontest = [float(label) 
                             for label in pred_file.readlines()]

In [206]:
sklearn.metrics.mean_absolute_error(valid_predictiontrain, traintarget.target), sklearn.metrics.mean_absolute_error(valid_predictiontest, testtarget.target)

(0.76644752783961712, 0.97655316005110981)

In [234]:
sklearn.metrics.mean_absolute_error(valid_predictiontrain, traintarget.target), sklearn.metrics.mean_absolute_error(valid_predictiontest, testtarget.target)

(0.76953773859212593, 0.96593903219355015)

In [210]:
sklearn.metrics.mean_absolute_error(valid_predictiontrain, traintarget.target), sklearn.metrics.mean_absolute_error(valid_predictiontest, testtarget.target)

(0.76772298677365203, 0.97901996766756472)

In [212]:
sklearn.metrics.mean_absolute_error(valid_predictiontrain, traintarget.target), sklearn.metrics.mean_absolute_error(valid_predictiontest, testtarget.target)

(0.77072359379278321, 0.9708573212410766)

In [236]:
sklearn.metrics.mean_absolute_error(valid_predictiontrain, traintarget.target), sklearn.metrics.mean_absolute_error(valid_predictiontest, testtarget.target)

(0.76953773859212593, 0.96593903219355015)

In [99]:
train_target2 = train_target.reset_index()

In [100]:
testtarget = train_target2[~train_target2.index.isin(myindices1)]

In [101]:
traintarget =  train_target2[train_target2.index.isin(myindices1)]

In [102]:
len(traintarget)

83999

In [197]:
sklearn.metrics.mean_absolute_error(valid_prediction, testtarget.target),  sklearn.metrics.mean_squared_error(valid_prediction, testtarget.target)

(0.96593903219355015, 1.4520665064748373)

In [319]:
with open('D:/habr_test_alt2_model1.txt') as pred_file:
    valid_predictiontest = [float(label) 
                             for label in pred_file.readlines()]

In [320]:
with open('D:/habr_train_alt2_model1.txt') as pred_file:
    valid_predictiontrain = [float(label) 
                             for label in pred_file.readlines()]

In [325]:
with open('D:/habr_train_alt2_model3.txt') as pred_file:
    valid_predictiontrain = [float(label) 
                             for label in pred_file.readlines()]

In [326]:
sklearn.metrics.mean_absolute_error(valid_predictiontrain, train_target.target)

0.89623898482499997

In [238]:
import pandas
len(pandas.DataFrame(valid_predictiontest))

52913

In [240]:
sample_sub.head()

,target
url,
https://geektimes.ru/post/87455/,11.620054
https://geektimes.ru/post/87452/,4.822528
https://geektimes.ru/post/87459/,0.921104
https://habrahabr.ru/post/87461/,1.632126
https://habrahabr.ru/post/5754/,1.952122


In [295]:
your_submission = sample_sub.copy()
your_submission['target'] = valid_predictiontest


In [323]:
your_submissionbi = sample_sub.copy()
your_submissionbi['target'] = valid_predictiontest

In [316]:
len(your_submissionbi)

52913

In [318]:
your_submissionbi.to_csv('D:/submission3.csv')

In [324]:
your_submission.describe(), train_target.describe(), your_submissionbi.describe()

(             target
 count  52913.000000
 mean       2.310127
 std        1.089067
 min        0.000000
 25%        1.497322
 50%        2.262579
 75%        3.095786
 max        9.080687,               target    antitarget
 count  120000.000000  1.200000e+05
 mean        2.828131  2.900000e+00
 std         1.696940  2.649447e-12
 min         0.000000  2.900000e+00
 25%         1.609438  2.900000e+00
 50%         2.944439  2.900000e+00
 75%         4.127134  2.900000e+00
 max         9.080687  2.900000e+00,              target
 count  52913.000000
 mean       2.310127
 std        1.089067
 min        0.000000
 25%        1.497322
 50%        2.262579
 75%        3.095786
 max        9.080687)

In [264]:
your_submission2 = your_submission

In [266]:
count = 0
with open('D:/test.json', 'r') as f:
    for line in f:
        traindat = json.loads(line)
        url = traindat['url']
        prob = valid_predictiontest[count]
        if count % 1000 == 0:
            print(count, prob, url, your_submission.target[your_submission.index == url])
        #break
        your_submission2.target[your_submission2.index == url] = prob
        count+=1

0 1.251891 https://geektimes.ru/post/87455/ url
https://geektimes.ru/post/87455/    1.251891
Name: target, dtype: float64
1000 4.312281 https://geektimes.ru/post/89204/ url
https://geektimes.ru/post/89204/    4.312281
Name: target, dtype: float64
2000 4.266988 https://habrahabr.ru/post/91055/ url
https://habrahabr.ru/post/91055/    4.266988
Name: target, dtype: float64
3000 6.103524 https://habrahabr.ru/post/92770/ url
https://habrahabr.ru/post/92770/    6.103524
Name: target, dtype: float64
4000 3.731129 https://habrahabr.ru/company/mailru/blog/94368/ url
https://habrahabr.ru/company/mailru/blog/94368/    3.731129
Name: target, dtype: float64
5000 4.928438 https://habrahabr.ru/post/95972/ url
https://habrahabr.ru/post/95972/    4.928438
Name: target, dtype: float64
6000 3.833031 https://habrahabr.ru/company/synchronet/blog/97644/ url
https://habrahabr.ru/company/synchronet/blog/97644/    3.833031
Name: target, dtype: float64
7000 3.746355 https://geektimes.ru/post/99189/ url
https://g

KeyboardInterrupt: 

In [255]:
sample_sub['https://geektimes.ru/post/87455/']

KeyError: 'https://geektimes.ru/post/87455/'

In [256]:
sample_sub.head()

,target
url,
https://geektimes.ru/post/87455/,11.620054
https://geektimes.ru/post/87452/,4.822528
https://geektimes.ru/post/87459/,0.921104
https://habrahabr.ru/post/87461/,1.632126
https://habrahabr.ru/post/5754/,1.952122


In [257]:
your_submission.head()

,target,https://geektimes.ru/post/87455/,https://geektimes.ru/post/87452/,https://geektimes.ru/post/87459/,https://habrahabr.ru/post/87461/,https://habrahabr.ru/post/5754/,https://geektimes.ru/post/87460/,https://geektimes.ru/post/87462/,https://habrahabr.ru/post/87467/,https://habrahabr.ru/post/87464/,...,https://geektimes.ru/post/113334/,https://habrahabr.ru/post/113332/,https://habrahabr.ru/post/113335/,https://geektimes.ru/post/7673/,https://geektimes.ru/company/asus/blog/261662/,https://habrahabr.ru/company/twins/blog/113336/,https://habrahabr.ru/post/113337/,https://geektimes.ru/post/113338/,https://habrahabr.ru/post/113339/,https://habrahabr.ru/post/113342/
url,,,,,,,,,,,,,,,,,,,,,
https://geektimes.ru/post/87455/,1.251891,1.251891,1.932414,1.477674,3.4071,0.0,4.250554,3.570882,4.106386,3.014565,...,4.754907,5.744237,5.117727,2.023568,5.347918,4.897154,1.498438,1.872094,3.130134,2.620596
https://geektimes.ru/post/87452/,1.932414,1.251891,1.932414,1.477674,3.4071,0.0,4.250554,3.570882,4.106386,3.014565,...,4.754907,5.744237,5.117727,2.023568,5.347918,4.897154,1.498438,1.872094,3.130134,2.620596
https://geektimes.ru/post/87459/,1.477674,1.251891,1.932414,1.477674,3.4071,0.0,4.250554,3.570882,4.106386,3.014565,...,4.754907,5.744237,5.117727,2.023568,5.347918,4.897154,1.498438,1.872094,3.130134,2.620596
https://habrahabr.ru/post/87461/,3.407100,1.251891,1.932414,1.477674,3.4071,0.0,4.250554,3.570882,4.106386,3.014565,...,4.754907,5.744237,5.117727,2.023568,5.347918,4.897154,1.498438,1.872094,3.130134,2.620596
https://habrahabr.ru/post/5754/,0.000000,1.251891,1.932414,1.477674,3.4071,0.0,4.250554,3.570882,4.106386,3.014565,...,4.754907,5.744237,5.117727,2.023568,5.347918,4.897154,1.498438,1.872094,3.130134,2.620596


In [258]:
your_submission3 = sample_sub.copy()
your_submission3['target'] = valid_predictiontest

In [259]:
(your_submission.target-your_submission3.target).describe()

count    52913.0
mean         0.0
std          0.0
min          0.0
25%          0.0
50%          0.0
75%          0.0
max          0.0
Name: target, dtype: float64

In [260]:
your_submission = your_submission3

In [261]:
your_submission[your_submission.index[0]]

KeyError: 'https://geektimes.ru/post/87455/'

In [281]:
your_submission.head(10)

,target
url,
https://geektimes.ru/post/87455/,1.251891
https://geektimes.ru/post/87452/,1.932414
https://geektimes.ru/post/87459/,1.477674
https://habrahabr.ru/post/87461/,3.407100
https://habrahabr.ru/post/5754/,0.000000
https://geektimes.ru/post/87460/,4.250554
https://geektimes.ru/post/87462/,3.570882
https://habrahabr.ru/post/87467/,4.106386
https://habrahabr.ru/post/87464/,3.014565


In [285]:
train_target.describe()
trr = train_target

In [286]:
train_target.mean(), train_target.median()

(target    2.828131
 dtype: float64, target    2.944439
 dtype: float64)

In [287]:
trr['antitarget'] = 2.9

In [288]:
sklearn.metrics.mean_absolute_error(trr.target, trr.antitarget)

1.4126459856166664

In [303]:
your_submission2.describe()

,target
count,52913.000000
mean,4.003422
std,1.343091
min,0.000000
25%,3.088488
50%,3.999583
75%,4.927497
max,9.080687


In [307]:
train_target.target.plot()
plt.show()

NameError: name 'plt' is not defined

In [97]:
def MiniCvScript(trainfile, testfile, traintab, testtab):
    with open(trainfile, 'r') as pred_file:
        valid_predictiontrain = [float(label) 
                             for label in pred_file.readlines()]
    with open(testfile, 'r') as pred_file:
        valid_predictiontest = [float(label) 
                             for label in pred_file.readlines()]
    print(sklearn.metrics.mean_absolute_error(valid_predictiontrain, traintab.target), sklearn.metrics.mean_absolute_error(valid_predictiontest, testtab.target))

In [333]:
MiniCvScript('D:/habr_minitrain_alt2_model.txt', 'D:/habr_minitest_alt2_model.txt', traintarget, testtarget)

0.843517558316 0.979019967668


In [348]:
MiniCvScript('D:/habr_minitrain_alt2_model_pt1.txt', 'D:/habr_minitest_alt2_model_pt1.txt', traintarget, testtarget)
MiniCvScript('D:/habr_minitrain_alt2_model_pt2.txt', 'D:/habr_minitest_alt2_model_pt2.txt', traintarget, testtarget)
MiniCvScript('D:/habr_minitrain_alt2_model_pt3.txt', 'D:/habr_minitest_alt2_model_pt3.txt', traintarget, testtarget)
MiniCvScript('D:/habr_minitrain_alt2_model_pt4.txt', 'D:/habr_minitest_alt2_model_pt4.txt', traintarget, testtarget)
MiniCvScript('D:/habr_minitrain_alt2_model_pt5.txt', 'D:/habr_minitest_alt2_model_pt5.txt', traintarget, testtarget)
MiniCvScript('D:/habr_minitrain_alt2_model_pt6.txt', 'D:/habr_minitest_alt2_model_pt6.txt', traintarget, testtarget)
MiniCvScript('D:/habr_minitrain_alt2_model_pt7.txt', 'D:/habr_minitest_alt2_model_pt7.txt', traintarget, testtarget)
MiniCvScript('D:/habr_minitrain_alt2_model_pt8.txt', 'D:/habr_minitest_alt2_model_pt8.txt', traintarget, testtarget)
MiniCvScript('D:/habr_minitrain_alt2_model_pt9.txt', 'D:/habr_minitest_alt2_model_pt9.txt', traintarget, testtarget)
MiniCvScript('D:/habr_minitrain_alt2_model_pt85.txt', 'D:/habr_minitest_alt2_model_pt85.txt', traintarget, testtarget)#!!!!

0.851518806688 0.958816180384
0.829707766616 0.947782659315
0.838839090834 0.96282187792
0.853342504042 0.982119794895
0.843517558316 0.979019967668
0.810883589709 0.955540020111
0.779618837879 0.93489977384
0.740431456613 0.910741492375
0.715175012429 0.911313081026
0.718366043512 0.89981409647


In [ ]:
#power_t = 0.85  is the best????

In [350]:
MiniCvScript('D:/habr_minitrain_alt2_model_pt85_lr1.txt', 'D:/habr_minitest_alt2_model_pt85_lr1.txt', traintarget, testtarget)
MiniCvScript('D:/habr_minitrain_alt2_model_pt85_lr2.txt', 'D:/habr_minitest_alt2_model_pt85_lr2.txt', traintarget, testtarget)
MiniCvScript('D:/habr_minitrain_alt2_model_pt85_lr3.txt', 'D:/habr_minitest_alt2_model_pt85_lr3.txt', traintarget, testtarget)
MiniCvScript('D:/habr_minitrain_alt2_model_pt85_lr4.txt', 'D:/habr_minitest_alt2_model_pt85_lr4.txt', traintarget, testtarget)
MiniCvScript('D:/habr_minitrain_alt2_model_pt85_lr5.txt', 'D:/habr_minitest_alt2_model_pt85_lr5.txt', traintarget, testtarget)
MiniCvScript('D:/habr_minitrain_alt2_model_pt85_lr6.txt', 'D:/habr_minitest_alt2_model_pt85_lr6.txt', traintarget, testtarget)
MiniCvScript('D:/habr_minitrain_alt2_model_pt85_lr7.txt', 'D:/habr_minitest_alt2_model_pt85_lr7.txt', traintarget, testtarget)
MiniCvScript('D:/habr_minitrain_alt2_model_pt85_lr8.txt', 'D:/habr_minitest_alt2_model_pt85_lr8.txt', traintarget, testtarget)
MiniCvScript('D:/habr_minitrain_alt2_model_pt85_lr9.txt', 'D:/habr_minitest_alt2_model_pt85_lr9.txt', traintarget, testtarget)
MiniCvScript('D:/habr_minitrain_alt2_model_pt85_lr55.txt', 'D:/habr_minitest_alt2_model_pt85_lr55.txt', traintarget, testtarget)


0.932412628817 1.00190528202
0.820752729342 0.926643218799
0.771529067334 0.909447423044
0.740200473148 0.902266524513
0.718366043512 0.89981409647
0.702066406386 0.899655122358
0.68894068677 0.900214108358
0.678042158609 0.901220746951
0.668999061489 0.902451043304
0.709689270003 0.899603524819


In [ ]:
#pt = 0.85, lr = 0.55

In [360]:
MiniCvScript('D:/habr_minitrain_alt2_model_pt85_lr55_b2.txt', 'D:/habr_minitest_alt2_model_pt85_lr55_b2.txt', traintarget, testtarget)
MiniCvScript('D:/habr_minitrain_alt2_model_pt85_lr55_b3.txt', 'D:/habr_minitest_alt2_model_pt85_lr55_b3.txt', traintarget, testtarget)
MiniCvScript('D:/habr_minitrain_alt2_model_pt85_lr55_b4.txt', 'D:/habr_minitest_alt2_model_pt85_lr55_b4.txt', traintarget, testtarget)
MiniCvScript('D:/habr_minitrain_alt2_model_pt85_lr55_b5.txt', 'D:/habr_minitest_alt2_model_pt85_lr55_b5.txt', traintarget, testtarget)
MiniCvScript('D:/habr_minitrain_alt2_model_pt85_lr55_b6.txt', 'D:/habr_minitest_alt2_model_pt85_lr55_b6.txt', traintarget, testtarget)
MiniCvScript('D:/habr_minitrain_alt2_model_pt85_lr55_b7.txt', 'D:/habr_minitest_alt2_model_pt85_lr55_b7.txt', traintarget, testtarget)
MiniCvScript('D:/habr_minitrain_alt2_model_pt85_lr55_b8.txt', 'D:/habr_minitest_alt2_model_pt85_lr55_b8.txt', traintarget, testtarget)
MiniCvScript('D:/habr_minitrain_alt2_model_pt85_lr55_b9.txt', 'D:/habr_minitest_alt2_model_pt85_lr55_b9.txt', traintarget, testtarget)
MiniCvScript('D:/habr_minitrain_alt2_model_pt85_lr55_b10.txt', 'D:/habr_minitest_alt2_model_pt85_lr55_b10.txt', traintarget, testtarget)
MiniCvScript('D:/habr_minitrain_alt2_model_pt85_lr55_b11.txt', 'D:/habr_minitest_alt2_model_pt85_lr55_b11.txt', traintarget, testtarget)
MiniCvScript('D:/habr_minitrain_alt2_model_pt85_lr55_b12.txt', 'D:/habr_minitest_alt2_model_pt85_lr55_b12.txt', traintarget, testtarget)
MiniCvScript('D:/habr_minitrain_alt2_model_pt85_lr55_b13.txt', 'D:/habr_minitest_alt2_model_pt85_lr55_b13.txt', traintarget, testtarget)
MiniCvScript('D:/habr_minitrain_alt2_model_pt85_lr55_b14.txt', 'D:/habr_minitest_alt2_model_pt85_lr55_b14.txt', traintarget, testtarget)
MiniCvScript('D:/habr_minitrain_alt2_model_pt85_lr55_b15.txt', 'D:/habr_minitest_alt2_model_pt85_lr55_b15.txt', traintarget, testtarget)
MiniCvScript('D:/habr_minitrain_alt2_model_pt85_lr55_b16.txt', 'D:/habr_minitest_alt2_model_pt85_lr55_b16.txt', traintarget, testtarget)
MiniCvScript('D:/habr_minitrain_alt2_model_pt85_lr55_b17.txt', 'D:/habr_minitest_alt2_model_pt85_lr55_b17.txt', traintarget, testtarget)
MiniCvScript('D:/habr_minitrain_alt2_model_pt85_lr55_b18.txt', 'D:/habr_minitest_alt2_model_pt85_lr55_b18.txt', traintarget, testtarget)
MiniCvScript('D:/habr_minitrain_alt2_model_pt85_lr55_b19.txt', 'D:/habr_minitest_alt2_model_pt85_lr55_b19.txt', traintarget, testtarget)
MiniCvScript('D:/habr_minitrain_alt2_model_pt85_lr55_b20.txt', 'D:/habr_minitest_alt2_model_pt85_lr55_b20.txt', traintarget, testtarget)
MiniCvScript('D:/habr_minitrain_alt2_model_pt85_lr55_b21.txt', 'D:/habr_minitest_alt2_model_pt85_lr55_b21.txt', traintarget, testtarget)
MiniCvScript('D:/habr_minitrain_alt2_model_pt85_lr55_b22.txt', 'D:/habr_minitest_alt2_model_pt85_lr55_b22.txt', traintarget, testtarget)
MiniCvScript('D:/habr_minitrain_alt2_model_pt85_lr55_b23.txt', 'D:/habr_minitest_alt2_model_pt85_lr55_b23.txt', traintarget, testtarget)
MiniCvScript('D:/habr_minitrain_alt2_model_pt85_lr55_b24.txt', 'D:/habr_minitest_alt2_model_pt85_lr55_b24.txt', traintarget, testtarget)
MiniCvScript('D:/habr_minitrain_alt2_model_pt85_lr55_b25.txt', 'D:/habr_minitest_alt2_model_pt85_lr55_b25.txt', traintarget, testtarget)
MiniCvScript('D:/habr_minitrain_alt2_model_pt85_lr55_b26.txt', 'D:/habr_minitest_alt2_model_pt85_lr55_b26.txt', traintarget, testtarget)
MiniCvScript('D:/habr_minitrain_alt2_model_pt85_lr55_b27.txt', 'D:/habr_minitest_alt2_model_pt85_lr55_b27.txt', traintarget, testtarget)
MiniCvScript('D:/habr_minitrain_alt2_model_pt85_lr55_b28.txt', 'D:/habr_minitest_alt2_model_pt85_lr55_b28.txt', traintarget, testtarget)
MiniCvScript('D:/habr_minitrain_alt2_model_pt85_lr55_b29.txt', 'D:/habr_minitest_alt2_model_pt85_lr55_b29.txt', traintarget, testtarget)


1.5475439007 1.55219324888
1.51274418503 1.51758066359
1.48912797127 1.49381907794
3.68714640289 3.6892509737
2.61139434965 2.62590668684
2.03918458159 2.05085485331
1.5369198618 1.54229153468
1.28037285265 1.29010235702
1.11690728582 1.13181472392
1.04364236053 1.06236815033
1.00322277099 1.03314192828
0.947848077846 0.991588086192
0.89861079089 0.968636755812
0.83548664927 0.942942241854
0.781276225479 0.926305259576
0.740450795081 0.913373181273
0.709689270003 0.899603524819
0.696900717211 0.894967809783
0.689962279491 0.892316342657
0.68665937185 0.89052796364
0.684097719913 0.889699957751
0.68304645341 0.888945022083
0.682446119835 0.88873948382
0.682148121228 0.888544792061
0.681897447601 0.888306670759
0.681892195729 0.888310560679
0.681864554495 0.888293639954
0.681878212074 0.888310367573


In [ ]:
#pt 0.85, lr 0.55, b 26

In [359]:
MiniCvScript('D:/habr_minitrain_alt2_model_pt85_lr55_b26_q.txt', 'D:/habr_minitest_alt2_model_pt85_lr55_b26_q.txt', traintarget, testtarget)
MiniCvScript('D:/habr_minitrain_alt2_model_pt85_lr55_b18_q.txt', 'D:/habr_minitest_alt2_model_pt85_lr55_b18_q.txt', traintarget, testtarget)


0.546619736199 0.891587938391
0.591757129918 0.914509784312


In [366]:
MiniCvScript('D:/habr_minitrain_alt2_model_pt85_lr1_b26.txt', 'D:/habr_minitest_alt2_model_pt85_lr1_b26.txt', traintarget, testtarget)
MiniCvScript('D:/habr_minitrain_alt2_model_pt85_lr2_b26.txt', 'D:/habr_minitest_alt2_model_pt85_lr2_b26.txt', traintarget, testtarget)
MiniCvScript('D:/habr_minitrain_alt2_model_pt85_lr3_b26.txt', 'D:/habr_minitest_alt2_model_pt85_lr3_b26.txt', traintarget, testtarget)
MiniCvScript('D:/habr_minitrain_alt2_model_pt85_lr4_b26.txt', 'D:/habr_minitest_alt2_model_pt85_lr4_b26.txt', traintarget, testtarget)
MiniCvScript('D:/habr_minitrain_alt2_model_pt85_lr5_b26.txt', 'D:/habr_minitest_alt2_model_pt85_lr5_b26.txt', traintarget, testtarget)
MiniCvScript('D:/habr_minitrain_alt2_model_pt85_lr6_b26.txt', 'D:/habr_minitest_alt2_model_pt85_lr6_b26.txt', traintarget, testtarget)
MiniCvScript('D:/habr_minitrain_alt2_model_pt85_lr7_b26.txt', 'D:/habr_minitest_alt2_model_pt85_lr7_b26.txt', traintarget, testtarget)
MiniCvScript('D:/habr_minitrain_alt2_model_pt85_lr8_b26.txt', 'D:/habr_minitest_alt2_model_pt85_lr8_b26.txt', traintarget, testtarget)
MiniCvScript('D:/habr_minitrain_alt2_model_pt85_lr9_b26.txt', 'D:/habr_minitest_alt2_model_pt85_lr9_b26.txt', traintarget, testtarget)


0.915657749949 0.995934988723
0.799033195598 0.91806811883
0.747241694104 0.899871101192
0.714226072715 0.892083194245
0.691146696925 0.888924083442
0.673703696996 0.888026120997
0.659552918213 0.887913585928
0.647757601019 0.88834396389
0.63788660296 0.889092091192


In [368]:
MiniCvScript('D:/habr_minitrain_alt2_model_pt1_lr7_b26.txt', 'D:/habr_minitest_alt2_model_pt1_lr7_b26.txt', traintarget, testtarget)
MiniCvScript('D:/habr_minitrain_alt2_model_pt2_lr7_b26.txt', 'D:/habr_minitest_alt2_model_pt2_lr7_b26.txt', traintarget, testtarget)
MiniCvScript('D:/habr_minitrain_alt2_model_pt3_lr7_b26.txt', 'D:/habr_minitest_alt2_model_pt3_lr7_b26.txt', traintarget, testtarget)
MiniCvScript('D:/habr_minitrain_alt2_model_pt4_lr7_b26.txt', 'D:/habr_minitest_alt2_model_pt4_lr7_b26.txt', traintarget, testtarget)
MiniCvScript('D:/habr_minitrain_alt2_model_pt5_lr7_b26.txt', 'D:/habr_minitest_alt2_model_pt5_lr7_b26.txt', traintarget, testtarget)
MiniCvScript('D:/habr_minitrain_alt2_model_pt6_lr7_b26.txt', 'D:/habr_minitest_alt2_model_pt6_lr7_b26.txt', traintarget, testtarget)
MiniCvScript('D:/habr_minitrain_alt2_model_pt7_lr7_b26.txt', 'D:/habr_minitest_alt2_model_pt7_lr7_b26.txt', traintarget, testtarget)
MiniCvScript('D:/habr_minitrain_alt2_model_pt8_lr7_b26.txt', 'D:/habr_minitest_alt2_model_pt8_lr7_b26.txt', traintarget, testtarget)
MiniCvScript('D:/habr_minitrain_alt2_model_pt9_lr7_b26.txt', 'D:/habr_minitest_alt2_model_pt9_lr7_b26.txt', traintarget, testtarget)
MiniCvScript('D:/habr_minitrain_alt2_model_pt85_lr7_b26.txt', 'D:/habr_minitest_alt2_model_pt85_lr7_b26.txt', traintarget, testtarget)


0.835826768926 0.959415923308
0.803048714104 0.942419239243
0.797997799176 0.949030076748
0.810380504863 0.969431193745
0.802778617031 0.971257895197
0.765245082132 0.946574985556
0.723594208788 0.919827184217
0.683549579055 0.89753579845
0.652632163026 0.902410404405
0.659552918213 0.887913585928


In [ ]:
####we try quantile loss function

In [377]:
MiniCvScript('D:/habr_minirain_alt2_model_pt1_lr5_b18_q.txt', 'D:/habr_minitest_alt2_model_pt1_lr5_b18_q.txt', traintarget, testtarget)
MiniCvScript('D:/habr_minirain_alt2_model_pt2_lr5_b18_q.txt', 'D:/habr_minitest_alt2_model_pt2_lr5_b18_q.txt', traintarget, testtarget)
MiniCvScript('D:/habr_minirain_alt2_model_pt3_lr5_b18_q.txt', 'D:/habr_minitest_alt2_model_pt3_lr5_b18_q.txt', traintarget, testtarget)
MiniCvScript('D:/habr_minirain_alt2_model_pt4_lr5_b18_q.txt', 'D:/habr_minitest_alt2_model_pt4_lr5_b18_q.txt', traintarget, testtarget)
MiniCvScript('D:/habr_minirain_alt2_model_pt5_lr5_b18_q.txt', 'D:/habr_minitest_alt2_model_pt5_lr5_b18_q.txt', traintarget, testtarget)
MiniCvScript('D:/habr_minirain_alt2_model_pt6_lr5_b18_q.txt', 'D:/habr_minitest_alt2_model_pt6_lr5_b18_q.txt', traintarget, testtarget)
MiniCvScript('D:/habr_minirain_alt2_model_pt7_lr5_b18_q.txt', 'D:/habr_minitest_alt2_model_pt7_lr5_b18_q.txt', traintarget, testtarget)
MiniCvScript('D:/habr_minirain_alt2_model_pt8_lr5_b18_q.txt', 'D:/habr_minitest_alt2_model_pt8_lr5_b18_q.txt', traintarget, testtarget)
MiniCvScript('D:/habr_minirain_alt2_model_pt9_lr5_b18_q.txt', 'D:/habr_minitest_alt2_model_pt9_lr5_b18_q.txt', traintarget, testtarget)
MiniCvScript('D:/habr_minirain_alt2_model_pt85_lr5_b18_q.txt', 'D:/habr_minitest_alt2_model_pt85_lr5_b18_q.txt', traintarget, testtarget)


0.906133904666 0.966965662093
0.893773106775 0.969067795895
0.890154294075 0.981856009611
0.87480823755 0.987831228466
0.828805071203 0.970857321241
0.768366795486 0.948115698953
0.692797305873 0.921172559484
0.622998288837 0.907675521041
0.575930682901 0.933626918724
0.593531435839 0.910480048582


In [378]:
MiniCvScript('D:/habr_minirain_alt2_model_pt8_lr1_b18_q.txt', 'D:/habr_minitest_alt2_model_pt8_lr1_b18_q.txt', traintarget, testtarget)
MiniCvScript('D:/habr_minirain_alt2_model_pt8_lr2_b18_q.txt', 'D:/habr_minitest_alt2_model_pt8_lr2_b18_q.txt', traintarget, testtarget)
MiniCvScript('D:/habr_minirain_alt2_model_pt8_lr3_b18_q.txt', 'D:/habr_minitest_alt2_model_pt8_lr3_b18_q.txt', traintarget, testtarget)
MiniCvScript('D:/habr_minirain_alt2_model_pt8_lr4_b18_q.txt', 'D:/habr_minitest_alt2_model_pt8_lr4_b18_q.txt', traintarget, testtarget)
MiniCvScript('D:/habr_minirain_alt2_model_pt8_lr5_b18_q.txt', 'D:/habr_minitest_alt2_model_pt8_lr5_b18_q.txt', traintarget, testtarget)
MiniCvScript('D:/habr_minirain_alt2_model_pt8_lr6_b18_q.txt', 'D:/habr_minitest_alt2_model_pt8_lr6_b18_q.txt', traintarget, testtarget)
MiniCvScript('D:/habr_minirain_alt2_model_pt8_lr7_b18_q.txt', 'D:/habr_minitest_alt2_model_pt8_lr7_b18_q.txt', traintarget, testtarget)
MiniCvScript('D:/habr_minirain_alt2_model_pt8_lr8_b18_q.txt', 'D:/habr_minitest_alt2_model_pt8_lr8_b18_q.txt', traintarget, testtarget)
MiniCvScript('D:/habr_minirain_alt2_model_pt8_lr9_b18_q.txt', 'D:/habr_minitest_alt2_model_pt8_lr9_b18_q.txt', traintarget, testtarget)


0.759054479768 0.887787159079
0.688018253741 0.883090560151
0.65291864046 0.890443121636
0.6329131658 0.898826274409
0.622998288837 0.907675521041
0.617358030405 0.914771389267
0.616080995833 0.921956786895
0.616180545887 0.928135429488
0.617767768271 0.933902734813


In [380]:
MiniCvScript('D:/habr_minirain_alt2_model_pt8_lr2_b13_q.txt', 'D:/habr_minitest_alt2_model_pt8_lr2_b13_q.txt', traintarget, testtarget)
MiniCvScript('D:/habr_minirain_alt2_model_pt8_lr2_b14_q.txt', 'D:/habr_minitest_alt2_model_pt8_lr2_b14_q.txt', traintarget, testtarget)
MiniCvScript('D:/habr_minirain_alt2_model_pt8_lr2_b15_q.txt', 'D:/habr_minitest_alt2_model_pt8_lr2_b15_q.txt', traintarget, testtarget)
MiniCvScript('D:/habr_minirain_alt2_model_pt8_lr2_b16_q.txt', 'D:/habr_minitest_alt2_model_pt8_lr2_b16_q.txt', traintarget, testtarget)
MiniCvScript('D:/habr_minirain_alt2_model_pt8_lr2_b17_q.txt', 'D:/habr_minitest_alt2_model_pt8_lr2_b17_q.txt', traintarget, testtarget)
MiniCvScript('D:/habr_minirain_alt2_model_pt8_lr2_b18_q.txt', 'D:/habr_minitest_alt2_model_pt8_lr2_b18_q.txt', traintarget, testtarget)
MiniCvScript('D:/habr_minirain_alt2_model_pt8_lr2_b19_q.txt', 'D:/habr_minitest_alt2_model_pt8_lr2_b19_q.txt', traintarget, testtarget)
MiniCvScript('D:/habr_minirain_alt2_model_pt8_lr2_b20_q.txt', 'D:/habr_minitest_alt2_model_pt8_lr2_b20_q.txt', traintarget, testtarget)
MiniCvScript('D:/habr_minirain_alt2_model_pt8_lr2_b21_q.txt', 'D:/habr_minitest_alt2_model_pt8_lr2_b21_q.txt', traintarget, testtarget)
MiniCvScript('D:/habr_minirain_alt2_model_pt8_lr2_b22_q.txt', 'D:/habr_minitest_alt2_model_pt8_lr2_b22_q.txt', traintarget, testtarget)
MiniCvScript('D:/habr_minirain_alt2_model_pt8_lr2_b23_q.txt', 'D:/habr_minitest_alt2_model_pt8_lr2_b23_q.txt', traintarget, testtarget)
MiniCvScript('D:/habr_minirain_alt2_model_pt8_lr2_b24_q.txt', 'D:/habr_minitest_alt2_model_pt8_lr2_b24_q.txt', traintarget, testtarget)

MiniCvScript('D:/habr_minirain_alt2_model_pt8_lr2_b25_q.txt', 'D:/habr_minitest_alt2_model_pt8_lr2_b25_q.txt', traintarget, testtarget)

MiniCvScript('D:/habr_minirain_alt2_model_pt8_lr2_b26_q.txt', 'D:/habr_minitest_alt2_model_pt8_lr2_b26_q.txt', traintarget, testtarget)

MiniCvScript('D:/habr_minirain_alt2_model_pt8_lr2_b27_q.txt', 'D:/habr_minitest_alt2_model_pt8_lr2_b27_q.txt', traintarget, testtarget)

MiniCvScript('D:/habr_minirain_alt2_model_pt8_lr2_b28_q.txt', 'D:/habr_minitest_alt2_model_pt8_lr2_b28_q.txt', traintarget, testtarget)


0.937916065096 0.9859732008
0.88512053053 0.960201406072
0.817257992309 0.926639487764
0.76479223586 0.911715026916
0.723198121787 0.900327336463
0.688018253741 0.883090560151
0.673689337921 0.878008161829
0.665095658294 0.87550942446
0.660635108287 0.873698891336
0.657634222657 0.872646986306
0.656681474125 0.872274708647
0.655802789021 0.872000577651
0.655596109644 0.871829290992
0.655249997179 0.871599600983
0.655252910094 0.871626783312
0.655244337159 0.871653101997


In [381]:
MiniCvScript('D:/habr_minirain_alt2_model_pt8_lr1_b26_q.txt', 'D:/habr_minitest_alt2_model_pt8_lr1_b26_q.txt', traintarget, testtarget)
MiniCvScript('D:/habr_minirain_alt2_model_pt8_lr2_b26_q.txt', 'D:/habr_minitest_alt2_model_pt8_lr2_b26_q.txt', traintarget, testtarget)
MiniCvScript('D:/habr_minirain_alt2_model_pt8_lr3_b26_q.txt', 'D:/habr_minitest_alt2_model_pt8_lr3_b26_q.txt', traintarget, testtarget)
MiniCvScript('D:/habr_minirain_alt2_model_pt8_lr4_b26_q.txt', 'D:/habr_minitest_alt2_model_pt8_lr4_b26_q.txt', traintarget, testtarget)
MiniCvScript('D:/habr_minirain_alt2_model_pt8_lr5_b26_q.txt', 'D:/habr_minitest_alt2_model_pt8_lr5_b26_q.txt', traintarget, testtarget)


0.734208838891 0.879210551985
0.655249997179 0.871599600983
0.616147487899 0.876029279464
0.593409391957 0.881232430432
0.580759462077 0.886604367823


In [398]:
MiniCvScript('D:/habr_minitrain_alt2_model_pt5_lr5_b18_nq.txt', 'D:/habr_minitest_alt2_model_pt5_lr5_b18_nq.txt', traintarget, testtarget)
MiniCvScript('D:/habr_minitrain_alt2_model_pt85_lr5_b26_nq.txt', 'D:/habr_minitest_alt2_model_pt85_lr5_b26_nq.txt', traintarget, testtarget)
MiniCvScript('D:/habr_minitrain_alt2_model_pt85_lr7_b26_nq.txt', 'D:/habr_minitest_alt2_model_pt85_lr7_b26_nq.txt', traintarget, testtarget)
MiniCvScript('D:/habr_minitrain_alt2_model_pt5_lr5_b18_nqq.txt', 'D:/habr_minitest_alt2_model_pt5_lr5_b18_nqq.txt', traintarget, testtarget)
MiniCvScript('D:/habr_minitrain_alt2_model_pt8_lr5_b26_nqq.txt', 'D:/habr_minitest_alt2_model_pt8_lr5_b26_nqq.txt', traintarget, testtarget)
MiniCvScript('D:/habr_minitrain_alt2_model_pt8_lr2_b26_nqq.txt', 'D:/habr_minitest_alt2_model_pt8_lr2_b26_nqq.txt', traintarget, testtarget)


0.803154344611 1.01239238643
0.536930358576 0.873726579067
0.508829278313 0.871303713952
0.79155594469 1.00715974203
0.428732842403 0.851956604317
0.465216684996 0.85129435874


In [400]:
MiniCvScript('D:/habr_minitrain_alt2_model_pt8_lr2_b29_nqq.txt', 'D:/habr_minitest_alt2_model_pt8_lr2_b29_nqq.txt', traintarget, testtarget)
MiniCvScript('D:/habr_minitrain_alt2_model_pt8_lr2_b28_nqq.txt', 'D:/habr_minitest_alt2_model_pt8_lr2_b28_nqq.txt', traintarget, testtarget)
MiniCvScript('D:/habr_minitrain_alt2_model_pt8_lr2_b27_nqq.txt', 'D:/habr_minitest_alt2_model_pt8_lr2_b27_nqq.txt', traintarget, testtarget)

MiniCvScript('D:/habr_minitrain_alt2_model_pt8_lr2_b26_nqq.txt', 'D:/habr_minitest_alt2_model_pt8_lr2_b26_nqq.txt', traintarget, testtarget)
MiniCvScript('D:/habr_minitrain_alt2_model_pt8_lr2_b25_nqq.txt', 'D:/habr_minitest_alt2_model_pt8_lr2_b25_nqq.txt', traintarget, testtarget)
MiniCvScript('D:/habr_minitrain_alt2_model_pt8_lr2_b24_nqq.txt', 'D:/habr_minitest_alt2_model_pt8_lr2_b24_nqq.txt', traintarget, testtarget)
MiniCvScript('D:/habr_minitrain_alt2_model_pt8_lr2_b23_nqq.txt', 'D:/habr_minitest_alt2_model_pt8_lr2_b23_nqq.txt', traintarget, testtarget)
MiniCvScript('D:/habr_minitrain_alt2_model_pt8_lr2_b22_nqq.txt', 'D:/habr_minitest_alt2_model_pt8_lr2_b22_nqq.txt', traintarget, testtarget)
MiniCvScript('D:/habr_minitrain_alt2_model_pt8_lr2_b21_nqq.txt', 'D:/habr_minitest_alt2_model_pt8_lr2_b21_nqq.txt', traintarget, testtarget)
MiniCvScript('D:/habr_minitrain_alt2_model_pt8_lr2_b20_nqq.txt', 'D:/habr_minitest_alt2_model_pt8_lr2_b20_nqq.txt', traintarget, testtarget)
MiniCvScript('D:/habr_minitrain_alt2_model_pt8_lr2_b19_nqq.txt', 'D:/habr_minitest_alt2_model_pt8_lr2_b19_nqq.txt', traintarget, testtarget)
MiniCvScript('D:/habr_minitrain_alt2_model_pt8_lr2_b18_nqq.txt', 'D:/habr_minitest_alt2_model_pt8_lr2_b18_nqq.txt', traintarget, testtarget)
MiniCvScript('D:/habr_minitrain_alt2_model_pt8_lr2_b17_nqq.txt', 'D:/habr_minitest_alt2_model_pt8_lr2_b17_nqq.txt', traintarget, testtarget)
MiniCvScript('D:/habr_minitrain_alt2_model_pt8_lr2_b16_nqq.txt', 'D:/habr_minitest_alt2_model_pt8_lr2_b16_nqq.txt', traintarget, testtarget)


0.464638269575 0.8512396179
0.464719496589 0.851366645899
0.464810977119 0.851260262993
0.465216684996 0.85129435874
0.465739541542 0.85150807697
0.467018106192 0.851921815894
0.469228841808 0.853502936391
0.475326536125 0.857154024722
0.488459048239 0.86293771948
0.507604157871 0.871925263854
0.584535784081 0.914643302103
0.614264910035 0.918328228355
0.692969413731 0.938800999222
0.782258580209 0.964871254854


In [402]:
MiniCvScript('D:/habr_minitrain_alt2_model_pt8_lr1_b29_nqq.txt', 'D:/habr_minitest_alt2_model_pt8_lr1_b29_nqq.txt', traintarget, testtarget)
MiniCvScript('D:/habr_minitrain_alt2_model_pt8_lr2_b29_nqq.txt', 'D:/habr_minitest_alt2_model_pt8_lr2_b29_nqq.txt', traintarget, testtarget)
MiniCvScript('D:/habr_minitrain_alt2_model_pt8_lr3_b29_nqq.txt', 'D:/habr_minitest_alt2_model_pt8_lr3_b29_nqq.txt', traintarget, testtarget)
MiniCvScript('D:/habr_minitrain_alt2_model_pt8_lr4_b29_nqq.txt', 'D:/habr_minitest_alt2_model_pt8_lr4_b29_nqq.txt', traintarget, testtarget)
MiniCvScript('D:/habr_minitrain_alt2_model_pt8_lr5_b29_nqq.txt', 'D:/habr_minitest_alt2_model_pt8_lr5_b29_nqq.txt', traintarget, testtarget)
MiniCvScript('D:/habr_minitrain_alt2_model_pt8_lr6_b29_nqq.txt', 'D:/habr_minitest_alt2_model_pt8_lr6_b29_nqq.txt', traintarget, testtarget)
MiniCvScript('D:/habr_minitrain_alt2_model_pt8_lr7_b29_nqq.txt', 'D:/habr_minitest_alt2_model_pt8_lr7_b29_nqq.txt', traintarget, testtarget)
MiniCvScript('D:/habr_minitrain_alt2_model_pt8_lr8_b29_nqq.txt', 'D:/habr_minitest_alt2_model_pt8_lr8_b29_nqq.txt', traintarget, testtarget)
MiniCvScript('D:/habr_minitrain_alt2_model_pt8_lr9_b29_nqq.txt', 'D:/habr_minitest_alt2_model_pt8_lr9_b29_nqq.txt', traintarget, testtarget)


0.541618674651 0.858054928863
0.464638269575 0.8512396179
0.441041368612 0.85137812658
0.43120456059 0.851086414933
0.428398172026 0.851919469765
0.427150841593 0.852453680314
0.426418119501 0.852805546846
0.4262582338 0.853155215494
0.424740725937 0.852597710258


In [403]:
MiniCvScript('D:/habr_minitrain_alt2_model_pt1_lr4_b29_nqq.txt', 'D:/habr_minitest_alt2_model_pt1_lr4_b29_nqq.txt', traintarget, testtarget)
MiniCvScript('D:/habr_minitrain_alt2_model_pt2_lr4_b29_nqq.txt', 'D:/habr_minitest_alt2_model_pt2_lr4_b29_nqq.txt', traintarget, testtarget)
MiniCvScript('D:/habr_minitrain_alt2_model_pt3_lr4_b29_nqq.txt', 'D:/habr_minitest_alt2_model_pt3_lr4_b29_nqq.txt', traintarget, testtarget)
MiniCvScript('D:/habr_minitrain_alt2_model_pt4_lr4_b29_nqq.txt', 'D:/habr_minitest_alt2_model_pt4_lr4_b29_nqq.txt', traintarget, testtarget)
MiniCvScript('D:/habr_minitrain_alt2_model_pt5_lr4_b29_nqq.txt', 'D:/habr_minitest_alt2_model_pt5_lr4_b29_nqq.txt', traintarget, testtarget)
MiniCvScript('D:/habr_minitrain_alt2_model_pt6_lr4_b29_nqq.txt', 'D:/habr_minitest_alt2_model_pt6_lr4_b29_nqq.txt', traintarget, testtarget)
MiniCvScript('D:/habr_minitrain_alt2_model_pt7_lr4_b29_nqq.txt', 'D:/habr_minitest_alt2_model_pt7_lr4_b29_nqq.txt', traintarget, testtarget)
MiniCvScript('D:/habr_minitrain_alt2_model_pt8_lr4_b29_nqq.txt', 'D:/habr_minitest_alt2_model_pt8_lr4_b29_nqq.txt', traintarget, testtarget)
MiniCvScript('D:/habr_minitrain_alt2_model_pt9_lr4_b29_nqq.txt', 'D:/habr_minitest_alt2_model_pt9_lr4_b29_nqq.txt', traintarget, testtarget)


0.915289580221 0.991404481792
0.899507220741 1.0014909335
0.865812942571 1.00292812747
0.808809769259 0.992069455404
0.730082275753 0.967960419488
0.632007824438 0.929818708064
0.531369521649 0.889676299131
0.43120456059 0.851086414933
0.348876589805 0.843791558818


In [438]:
MiniCvScript('D:/habr_minitrain_alt2_model_pt9_lr4_b29_nqq.txt', 'D:/habr_minitest_alt2_model_pt9_lr4_b29_nqq.txt', traintarget, testtarget)
MiniCvScript('D:/habr_minitrain_alt2_model_pt95_lr4_b29_nqq.txt', 'D:/habr_minitest_alt2_model_pt95_lr4_b29_nqq.txt', traintarget, testtarget)
MiniCvScript('D:/habr_minitrain_alt2_model_pt99_lr4_b29_nqq.txt', 'D:/habr_minitest_alt2_model_pt99_lr4_b29_nqq.txt', traintarget, testtarget)
MiniCvScript('D:/habr_minitrain_alt2_model_pt85_lr4_b29_nqq.txt', 'D:/habr_minitest_alt2_model_pt85_lr4_b29_nqq.txt', traintarget, testtarget)
MiniCvScript('D:/habr_minitrain_alt2_model_pt88_lr4_b29_nqq.txt', 'D:/habr_minitest_alt2_model_pt88_lr4_b29_nqq.txt', traintarget, testtarget)


0.348876589805 0.843791558818
0.328458024917 0.895720771951
0.338778588233 1.01569983767
0.386285446898 0.838974204828
0.362551134275 0.838440237938


In [439]:
MiniCvScript('D:/habr_minitrain_alt2_model_pt85_lr2_b29_nqq.txt', 'D:/habr_minitest_alt2_model_pt85_lr2_b29_nqq.txt', traintarget, testtarget)
MiniCvScript('D:/habr_minitrain_alt2_model_pt85_lr3_b29_nqq.txt', 'D:/habr_minitest_alt2_model_pt85_lr3_b29_nqq.txt', traintarget, testtarget)
MiniCvScript('D:/habr_minitrain_alt2_model_pt85_lr4_b29_nqq.txt', 'D:/habr_minitest_alt2_model_pt85_lr4_b29_nqq.txt', traintarget, testtarget)
MiniCvScript('D:/habr_minitrain_alt2_model_pt85_lr5_b29_nqq.txt', 'D:/habr_minitest_alt2_model_pt85_lr5_b29_nqq.txt', traintarget, testtarget)


0.408110142002 0.838352287242
0.390788014595 0.83817220855
0.386285446898 0.838974204828
0.38479129467 0.839535323935


In [440]:
MiniCvScript('D:/habr_minitrain_alt2_model_pt8_lr3_b29_nqq.txt', 'D:/habr_minitest_alt2_model_pt8_lr3_b29_nqq.txt', traintarget, testtarget)
MiniCvScript('D:/habr_minitrain_alt2_model_pt85_lr3_b29_nqq.txt', 'D:/habr_minitest_alt2_model_pt85_lr3_b29_nqq.txt', traintarget, testtarget)
MiniCvScript('D:/habr_minitrain_alt2_model_pt9_lr3_b29_nqq.txt', 'D:/habr_minitest_alt2_model_pt9_lr3_b29_nqq.txt', traintarget, testtarget)


0.441041368612 0.85137812658
0.390788014595 0.83817220855
0.351460051429 0.843102760173


In [447]:
MiniCvScript('D:/habr_minitrain_alt2_model_pt85_lr3_b26_nqq.txt', 'D:/habr_minitest_alt2_model_pt85_lr3_b26_nqq.txt', traintarget, testtarget)
MiniCvScript('D:/habr_minitrain_alt2_model_pt85_lr3_b28_nqq.txt', 'D:/habr_minitest_alt2_model_pt85_lr3_b28_nqq.txt', traintarget, testtarget)
MiniCvScript('D:/habr_minitrain_alt2_model_pt85_lr3_b29_nqq.txt', 'D:/habr_minitest_alt2_model_pt85_lr3_b29_nqq.txt', traintarget, testtarget)
MiniCvScript('D:/habr_minitrain_alt2_model_pt85_lr3_b30_nqq.txt', 'D:/habr_minitest_alt2_model_pt85_lr3_b30_nqq.txt', traintarget, testtarget)


0.39110227761 0.838212425655
0.390843321897 0.8383124091
0.390788014595 0.83817220855
0.39075824117 0.838173423599


In [448]:
MiniCvScript('D:/habr_minitrain_alt2_model_pt81_lr3_b29_nqq.txt', 'D:/habr_minitest_alt2_model_pt81_lr3_b29_nqq.txt', traintarget, testtarget)
MiniCvScript('D:/habr_minitrain_alt2_model_pt82_lr3_b29_nqq.txt', 'D:/habr_minitest_alt2_model_pt82_lr3_b29_nqq.txt', traintarget, testtarget)
MiniCvScript('D:/habr_minitrain_alt2_model_pt83_lr3_b29_nqq.txt', 'D:/habr_minitest_alt2_model_pt83_lr3_b29_nqq.txt', traintarget, testtarget)
MiniCvScript('D:/habr_minitrain_alt2_model_pt84_lr3_b29_nqq.txt', 'D:/habr_minitest_alt2_model_pt84_lr3_b29_nqq.txt', traintarget, testtarget)
MiniCvScript('D:/habr_minitrain_alt2_model_pt85_lr3_b29_nqq.txt', 'D:/habr_minitest_alt2_model_pt85_lr3_b29_nqq.txt', traintarget, testtarget)
MiniCvScript('D:/habr_minitrain_alt2_model_pt86_lr3_b29_nqq.txt', 'D:/habr_minitest_alt2_model_pt86_lr3_b29_nqq.txt', traintarget, testtarget)
MiniCvScript('D:/habr_minitrain_alt2_model_pt87_lr3_b29_nqq.txt', 'D:/habr_minitest_alt2_model_pt87_lr3_b29_nqq.txt', traintarget, testtarget)
MiniCvScript('D:/habr_minitrain_alt2_model_pt88_lr3_b29_nqq.txt', 'D:/habr_minitest_alt2_model_pt88_lr3_b29_nqq.txt', traintarget, testtarget)
MiniCvScript('D:/habr_minitrain_alt2_model_pt89_lr3_b29_nqq.txt', 'D:/habr_minitest_alt2_model_pt89_lr3_b29_nqq.txt', traintarget, testtarget)


0.430446199848 0.847979060943
0.420012172943 0.844874022388
0.409845276741 0.842101832699
0.400045230015 0.839818455182
0.390788014595 0.83817220855
0.382082546435 0.837199339518
0.373684001214 0.83696554529
0.365715617186 0.837712145468
0.358237351207 0.839693170079


In [451]:
MiniCvScript('D:/habr_minitrain_alt2_model_pt87_lr1_b29_nqq.txt', 'D:/habr_minitest_alt2_model_pt87_lr1_b29_nqq.txt', traintarget, testtarget)
MiniCvScript('D:/habr_minitrain_alt2_model_pt87_lr2_b29_nqq.txt', 'D:/habr_minitest_alt2_model_pt87_lr2_b29_nqq.txt', traintarget, testtarget)
MiniCvScript('D:/habr_minitrain_alt2_model_pt87_lr3_b29_nqq.txt', 'D:/habr_minitest_alt2_model_pt87_lr3_b29_nqq.txt', traintarget, testtarget)
MiniCvScript('D:/habr_minitrain_alt2_model_pt87_lr4_b29_nqq.txt', 'D:/habr_minitest_alt2_model_pt87_lr4_b29_nqq.txt', traintarget, testtarget)
MiniCvScript('D:/habr_minitrain_alt2_model_pt87_lr17_b29_nqq.txt', 'D:/habr_minitest_alt2_model_pt87_lr17_b29_nqq.txt', traintarget, testtarget)
####pt 87, lr 2, b 29

0.452943350992 0.839853877892
0.388281892201 0.836827084998
0.373684001214 0.83696554529
0.370078662734 0.837639982889
0.397996333218 0.836877904947


In [533]:
MiniCvScript('D:/habr_minitrain_alt2_model_pt5_lr5_b18_inter.txt', 'D:/habr_minitest_alt2_model_pt5_lr5_b18_inter.txt', traintarget, testtarget)
MiniCvScript('D:/habr_minitrain_alt2_model_pt87_lr2_b29_inter.txt', 'D:/habr_minitest_alt2_model_pt87_lr2_b29_inter.txt', traintarget, testtarget)


1.01000921804 1.11920563945
0.362084870689 1.23751834868


In [8]:
def MiniCvScriptPasses(trainfile, testfile, traintab, testtab, numpasses = 4, control = 1):
    with open(trainfile, 'r') as pred_file:
        valid_predictiontrain = [float(label) 
                             for label in pred_file.readlines()]
    with open(testfile, 'r') as pred_file:
        valid_predictiontest = [float(label) 
                             for label in pred_file.readlines()]
    valid_predictiontrain = np.array(valid_predictiontrain)
    valid_predictiontest = np.array(valid_predictiontest)
    mlen = int(len(valid_predictiontrain)/numpasses)
    trainarr = valid_predictiontrain[0:mlen]
    testarr = valid_predictiontest[0:mlen]
    for j in range(1,numpasses):
        trainarr += valid_predictiontrain[j*mlen:(j+1)*mlen]
    trainarr = trainarr/4
    mlen = int(len(valid_predictiontest)/numpasses)    
    testarr = valid_predictiontest[0:mlen]
    for j in range(1,numpasses):
        testarr += valid_predictiontest[j*mlen:(j+1)*mlen]
    testarr = testarr/4

    print(len(trainarr))
    print(sklearn.metrics.mean_absolute_error(trainarr, traintab.target))
    print(sklearn.metrics.mean_absolute_error(testarr, testtab.target))
    if control == 0:
        return(trainarr, testarr)
    
    #print(sklearn.metrics.mean_absolute_error(valid_predictiontrain, traintab.target), sklearn.metrics.mean_absolute_error(valid_predictiontest, testtab.target))    

In [118]:
MiniCvScript('D:/habr_mymodel_p5l5b29minitrain.txt', 'D:/habr_mymodel_p5l5b29minitest.txt', traintarget, testtarget)
MiniCvScript('D:/habr_mymodel_p8l2b29minitrain.txt', 'D:/habr_mymodel_p8l2b29minitest.txt', traintarget, testtarget)
MiniCvScript('D:/habr_mymodel_p88l17b29minitrain.txt', 'D:/habr_mymodel_p88l17b29minitest.txt', traintarget, testtarget)
MiniCvScript('D:/habr_mymodel_p91l17b29minitrain.txt', 'D:/habr_mymodel_p91l17b29minitest.txt', traintarget, testtarget)
MiniCvScript('D:/habr_mymodel_p92l17b29minitrain.txt', 'D:/habr_mymodel_p92l17b29minitest.txt', traintarget, testtarget)
MiniCvScript('D:/habr_mymodel_p93l17b29minitrain.txt', 'D:/habr_mymodel_p93l17b29minitest.txt', traintarget, testtarget)
MiniCvScript('D:/habr_mymodel_p94l17b29minitrain.txt', 'D:/habr_mymodel_p94l17b29minitest.txt', traintarget, testtarget)
MiniCvScript('D:/habr_mymodel_p95l17b29minitrain.txt', 'D:/habr_mymodel_p95l17b29minitest.txt', traintarget, testtarget)


0.536478617829 0.855776675231
0.363819803391 0.80208172498
0.305860595817 0.796296677203
0.277169678865 0.79597841685
0.0991036716866 0.787864473015
0.0959156351385 0.790322591595
0.0935635086727 0.792999394989
0.091873384576 0.795863911586


In [152]:
MiniCvScript('D:/habr_mymodel_p92l18b29minitrain__3.txt', 'D:/habr_mymodel_p92l18b29minitest__3.txt', traintarget, testtarget)
MiniCvScript('D:/habr_mymodel_p92l18b29minitrain__4.txt', 'D:/habr_mymodel_p92l18b29minitest__4.txt', traintarget, testtarget)
MiniCvScript('D:/habr_mymodel_p92l18b29minitrain__5.txt', 'D:/habr_mymodel_p92l18b29minitest__5.txt', traintarget, testtarget)


0.173259682532 0.784900663426
0.13505359315 0.786998233327
0.127734272718 0.789862174856


In [129]:
MiniCvScript('D:/habr_mymodel_p92l16b29minitrain.txt', 'D:/habr_mymodel_p92l16b29minitest.txt', traintarget, testtarget)
MiniCvScript('D:/habr_mymodel_p92l17b29minitrain.txt', 'D:/habr_mymodel_p92l17b29minitest.txt', traintarget, testtarget)
MiniCvScript('D:/habr_mymodel_p92l18b29minitrain.txt', 'D:/habr_mymodel_p92l18b29minitest.txt', traintarget, testtarget)
MiniCvScript('D:/habr_mymodel_p92l2b29minitrain.txt', 'D:/habr_mymodel_p92l2b29minitest.txt', traintarget, testtarget)
MiniCvScript('D:/habr_mymodel_p92l18b26minitrain.txt', 'D:/habr_mymodel_p92l18b26minitest.txt', traintarget, testtarget)
#92, 18, 29
MiniCvScript('D:/habr_mymodel_p92l18b29_3minitrain.txt', 'D:/habr_mymodel_p92l18b29_3minitest.txt', traintarget, testtarget)
MiniCvScript('D:/habr_mymodel_p9l2b29_3minitrain.txt', 'D:/habr_mymodel_p9l2b29_3minitest.txt', traintarget, testtarget)
MiniCvScript('D:/habr_mymodel_p88l17b29_3minitrain.txt', 'D:/habr_mymodel_p88l17b29_3minitest.txt', traintarget, testtarget)
MiniCvScript('D:/habr_mymodel_p86l17b29_3minitrain.txt', 'D:/habr_mymodel_p86l17b29_3minitest.txt', traintarget, testtarget)
MiniCvScript('D:/habr_mymodel_p85l17b29_3minitrain.txt', 'D:/habr_mymodel_p85l17b29_3minitest.txt', traintarget, testtarget)
MiniCvScript('D:/habr_mymodel_p84l17b29_3minitrain.txt', 'D:/habr_mymodel_p84l17b29_3minitest.txt', traintarget, testtarget)
MiniCvScript('D:/habr_mymodel_p83l17b29_3minitrain.txt', 'D:/habr_mymodel_p83l17b29_3minitest.txt', traintarget, testtarget)
MiniCvScript('D:/habr_mymodel_p8l17b29_3minitrain.txt', 'D:/habr_mymodel_p8l17b29_3minitest.txt', traintarget, testtarget)

MiniCvScript('D:/habr_mymodel_p85l16b29_3minitrain.txt', 'D:/habr_mymodel_p85l16b29_3minitest.txt', traintarget, testtarget)
MiniCvScript('D:/habr_mymodel_p85l18b29_3minitrain.txt', 'D:/habr_mymodel_p85l18b29_3minitest.txt', traintarget, testtarget)


0.099423349147 0.787886048749
0.0991036716866 0.787864473015
0.0988455353278 0.787855334407
0.0985334101954 0.787896855087
0.0991767834974 0.787910063582
0.0947376089477 0.790700477515
0.0903519865236 0.785051232188
0.0967422421695 0.781564756368
0.0949880081668 0.777241027916
0.0992719926428 0.776007448821
0.282582284599 0.788495667426
0.292775145359 0.789999838254
0.320165159276 0.795100609289
0.099423464577 0.776142834449
0.0991575267444 0.775885884948


In [154]:
MiniCvScript('D:/habr_mymodel_p85l3b29minitrain_33.txt', 'D:/habr_mymodel_p85l3b29minitest_33.txt', traintarget, testtarget)
MiniCvScript('D:/habr_mymodel_p85l3b29minitrain_34.txt', 'D:/habr_mymodel_p85l3b29minitest_34.txt', traintarget, testtarget)
MiniCvScript('D:/habr_mymodel_p85l3b29minitrain_35.txt', 'D:/habr_mymodel_p85l3b29minitest_35.txt', traintarget, testtarget)


0.174796904463 0.77248148432
0.136339436493 0.774397152551
0.132869135573 0.77844062376


In [170]:
MiniCvScript('D:/habr_mymodel_p85l2b29minitrain_2.txt', 'D:/habr_mymodel_p85l2b29minitest_2.txt', traintarget, testtarget)
MiniCvScript('D:/habr_mymodel_p85l3b29minitrain_21.txt', 'D:/habr_mymodel_p85l3b29minitest_21.txt', traintarget, testtarget)

0.175818332242 0.772720361907
0.265214279444 0.786030417016


In [175]:
MiniCvScript('D:/habr_mymodel_p8l3b29minitrain_2.txt', 'D:/habr_mymodel_p8l3b29minitest_2.txt', traintarget, testtarget)
MiniCvScript('D:/habr_mymodel_p82l3b29minitrain_2.txt', 'D:/habr_mymodel_p82l3b29minitest_2.txt', traintarget, testtarget)

MiniCvScript('D:/habr_mymodel_p83l3b29minitrain_2.txt', 'D:/habr_mymodel_p83l3b29minitest_2.txt', traintarget, testtarget)
MiniCvScript('D:/habr_mymodel_p84l3b29minitrain_2.txt', 'D:/habr_mymodel_p84l3b29minitest_2.txt', traintarget, testtarget)
MiniCvScript('D:/habr_mymodel_p85l3b29minitrain_2.txt', 'D:/habr_mymodel_p85l3b29minitest_2.txt', traintarget, testtarget)
MiniCvScript('D:/habr_mymodel_p86l3b29minitrain_2.txt', 'D:/habr_mymodel_p86l3b29minitest_2.txt', traintarget, testtarget)


0.240045995071 0.775120699092
0.212690585102 0.772342450126
0.199271829748 0.771783842477
0.18651464977 0.771836138691
0.174796904463 0.77248148432
0.16432990488 0.773710131913


In [178]:
MiniCvScript('D:/habr_mymodel_p83l25b29minitrain_2.txt', 'D:/habr_mymodel_p83l25b29minitest_2.txt', traintarget, testtarget)
MiniCvScript('D:/habr_mymodel_p83l3b29minitrain_2.txt', 'D:/habr_mymodel_p83l3b29minitest_2.txt', traintarget, testtarget)
MiniCvScript('D:/habr_mymodel_p83l35b29minitrain_2.txt', 'D:/habr_mymodel_p83l35b29minitest_2.txt', traintarget, testtarget)
MiniCvScript('D:/habr_mymodel_p83l4b29minitrain_2.txt', 'D:/habr_mymodel_p83l4b29minitest_2.txt', traintarget, testtarget)


MiniCvScript('D:/habr_mymodel_p82l35b29minitrain_2.txt', 'D:/habr_mymodel_p82l35b29minitest_2.txt', traintarget, testtarget)
MiniCvScript('D:/habr_mymodel_p83l35b29minitrain_2.txt', 'D:/habr_mymodel_p83l35b29minitest_2.txt', traintarget, testtarget)
MiniCvScript('D:/habr_mymodel_p84l35b29minitrain_2.txt', 'D:/habr_mymodel_p84l35b29minitest_2.txt', traintarget, testtarget)
#pt 83 lr 35 b 29

0.199601060429 0.77178627827
0.199271829748 0.771783842477
0.199281780081 0.771779607566
0.199342615936 0.771802585095
0.212621209955 0.772356953029
0.199281780081 0.771779607566
0.186557059155 0.77181514419


In [145]:
MiniCvScript('D:/habr_mymodel_p85l2b29minitrain_3.txt', 'D:/habr_mymodel_p85l2b29minitest_3.txt', traintarget, testtarget)
MiniCvScript('D:/habr_mymodel_p85l25b29minitrain_3.txt', 'D:/habr_mymodel_p85l25b29minitest_3.txt', traintarget, testtarget)
MiniCvScript('D:/habr_mymodel_p85l3b29minitrain_3.txt', 'D:/habr_mymodel_p85l3b29minitest_3.txt', traintarget, testtarget)
MiniCvScript('D:/habr_mymodel_p85l4b29minitrain_3.txt', 'D:/habr_mymodel_p85l4b29minitest_3.txt', traintarget, testtarget)

MiniCvScript('D:/habr_mymodel_p84l2b29minitrain_3.txt', 'D:/habr_mymodel_p84l2b29minitest_3.txt', traintarget, testtarget)
MiniCvScript('D:/habr_mymodel_p85l2b29minitrain_3.txt', 'D:/habr_mymodel_p85l2b29minitest_3.txt', traintarget, testtarget)
MiniCvScript('D:/habr_mymodel_p86l2b29minitrain_3.txt', 'D:/habr_mymodel_p86l2b29minitest_3.txt', traintarget, testtarget)


0.0990373749092 0.775726209883
0.0990261206324 0.775589931835
0.0990392274789 0.775647915586
0.0990490233574 0.775701778562
0.279858554066 0.787965578289
0.0990373749092 0.775726209883


FileNotFoundError: [Errno 2] No such file or directory: 'D:/habr_mymodel_p86l2b29minitest_3.txt'

In [132]:
MiniCvScript('D:/habr_mymodel_p2l2b29minitrain.txt', 'D:/habr_mymodel_p2l2b29minitest.txt', traintarget, testtarget)
MiniCvScript('D:/habr_mymodel_p2l5b29minitrain.txt', 'D:/habr_mymodel_p2l5b29minitest.txt', traintarget, testtarget)
MiniCvScript('D:/habr_mymodel_p2l8b29minitrain.txt', 'D:/habr_mymodel_p2l8b29minitest.txt', traintarget, testtarget)


0.59220399494 0.852107547318
0.589836088418 0.855399264937
0.589313884046 0.853342954918


In [499]:
#MiniCvScriptPasses('D:/habr_minitrain_alt2_model_pt87_lr2_b29_nqq.txt', 'D:/habr_minitest_alt2_model_pt87_lr2_b29_nqq.txt', traintarget, testtarget)
MiniCvScriptPasses('D:/habr_minitrain_alt2_model_pt87_lr2_b29_nqqp25.txt', 'D:/habr_minitest_alt2_model_pt87_lr2_b29_nqqp25.txt', traintarget, testtarget)
MiniCvScriptPasses('D:/habr_minitrain_alt2_model_pt87_lr2_b29_nqqp25p.txt', 'D:/habr_minitest_alt2_model_pt87_lr2_b29_nqqp25p.txt', traintarget, testtarget)
MiniCvScriptPasses('D:/habr_minitrain_alt2_model_pt87_lr2_b29_nqqp25p4.txt', 'D:/habr_minitest_alt2_model_pt87_lr2_b29_nqqp25p4.txt', traintarget, testtarget)


83999
0.30008459934
0.811825629871
83999
0.352096406886
0.813318379823
83999
0.381833664817
0.817195168134


In [539]:
MiniCvScriptPasses('D:/habr_minitrain_alt2_model_pt87_lr2_b29_nqqp.txt', 'D:/habr_minitest_alt2_model_pt87_lr2_b29_nqqp.txt', traintarget, testtarget)
MiniCvScriptPasses('D:/habr_minitrain_alt2_model_pt87_lr2_b29_nqqp2.txt', 'D:/habr_minitest_alt2_model_pt87_lr2_b29_nqqp2.txt', traintarget, testtarget)
MiniCvScript('D:/habr_minitrain_alt2_model_pt87_lr2_b29_nqqp3.txt', 'D:/habr_minitest_alt2_model_pt87_lr2_b29_nqqp3.txt', traintarget, testtarget)
MiniCvScriptPasses('D:/habr_minitrain_alt0_model_pt87_lr2_b29_nqqp.txt', 'D:/habr_minitest_alt0_model_pt87_lr2_b29_nqqp.txt', traintarget, testtarget)


83999
0.352096406886
0.813318379823
83999
0.352096406886
0.813318379823
0.352096406886 0.813318379823
83999
0.36799914974
0.809379125191


In [544]:
MiniCvScriptPasses('D:/habr_minitrain_alt0_model_pt87_lr2_b29_nqqp.txt', 'D:/habr_minitest_alt0_model_pt87_lr2_b29_nqqp.txt', traintarget, testtarget)
MiniCvScriptPasses('D:/habr_minitrain_alt0_model_pt87_lr2_b26_nqqp.txt', 'D:/habr_minitest_alt0_model_pt87_lr2_b26_nqqp.txt', traintarget, testtarget)
MiniCvScriptPasses('D:/habr_minitrain_alt0_model_pt87_lr2_b20_nqqp.txt', 'D:/habr_minitest_alt0_model_pt87_lr2_b20_nqqp.txt', traintarget, testtarget)
MiniCvScriptPasses('D:/habr_minitrain_alt0_model_pt87_lr2_b18_nqqp.txt', 'D:/habr_minitest_alt0_model_pt87_lr2_b18_nqqp.txt', traintarget, testtarget)
MiniCvScriptPasses('D:/habr_minitrain_alt0_model_pt87_lr2_b12_nqqp.txt', 'D:/habr_minitest_alt0_model_pt87_lr2_b12_nqqp.txt', traintarget, testtarget)


83999
0.36799914974
0.809379125191
83999
0.368497834093
0.809579064637
83999
0.404512933916
0.837628504375
83999
0.61694526792
0.915670396572
83999
0.989075800676
1.03270844907


In [545]:
MiniCvScriptPasses('D:/habr_minitrain_alt0_model_pt5_lr2_b29_nqqp.txt', 'D:/habr_minitest_alt0_model_pt5_lr2_b29_nqqp.txt', traintarget, testtarget)
MiniCvScriptPasses('D:/habr_minitrain_alt0_model_pt7_lr2_b29_nqqp.txt', 'D:/habr_minitest_alt0_model_pt7_lr2_b29_nqqp.txt', traintarget, testtarget)
MiniCvScriptPasses('D:/habr_minitrain_alt0_model_pt8_lr2_b29_nqqp.txt', 'D:/habr_minitest_alt0_model_pt8_lr2_b29_nqqp.txt', traintarget, testtarget)
MiniCvScriptPasses('D:/habr_minitrain_alt0_model_pt85_lr2_b29_nqqp.txt', 'D:/habr_minitest_alt0_model_pt85_lr2_b29_nqqp.txt', traintarget, testtarget)
MiniCvScriptPasses('D:/habr_minitrain_alt0_model_pt87_lr2_b29_nqqp.txt', 'D:/habr_minitest_alt0_model_pt87_lr2_b29_nqqp.txt', traintarget, testtarget)
MiniCvScriptPasses('D:/habr_minitrain_alt0_model_pt9_lr2_b29_nqqp.txt', 'D:/habr_minitest_alt0_model_pt9_lr2_b29_nqqp.txt', traintarget, testtarget)


83999
0.593823506566
0.876449218327
83999
0.519225476994
0.850290262687
83999
0.42446562046
0.822002975278
83999
0.384021979643
0.81179311308
83999
0.36799914974
0.809379125191
83999
0.344741902023
0.810610361073


In [546]:
MiniCvScriptPasses('D:/habr_minitrain_alt0_model_pt87_lr1_b29_nqqp.txt', 'D:/habr_minitest_alt0_model_pt87_lr1_b29_nqqp.txt', traintarget, testtarget)
MiniCvScriptPasses('D:/habr_minitrain_alt0_model_pt87_lr2_b29_nqqp.txt', 'D:/habr_minitest_alt0_model_pt87_lr2_b29_nqqp.txt', traintarget, testtarget)
MiniCvScriptPasses('D:/habr_minitrain_alt0_model_pt87_lr3_b29_nqqp.txt', 'D:/habr_minitest_alt0_model_pt87_lr3_b29_nqqp.txt', traintarget, testtarget)
MiniCvScriptPasses('D:/habr_minitrain_alt0_model_pt87_lr4_b29_nqqp.txt', 'D:/habr_minitest_alt0_model_pt87_lr4_b29_nqqp.txt', traintarget, testtarget)
MiniCvScriptPasses('D:/habr_minitrain_alt0_model_pt87_lr5_b29_nqqp.txt', 'D:/habr_minitest_alt0_model_pt87_lr5_b29_nqqp.txt', traintarget, testtarget)


83999
0.406179921332
0.811325897475
83999
0.36799914974
0.809379125191
83999
0.361233522804
0.810187336491
83999
0.358970397505
0.810423821866
83999
0.358633259051
0.810663485125


In [547]:
MiniCvScriptPasses('D:/habr_minitrain_alt0_model_pt85_lr2_b29_nqqp.txt', 'D:/habr_minitest_alt0_model_pt85_lr2_b29_nqqp.txt', traintarget, testtarget)
MiniCvScriptPasses('D:/habr_minitrain_alt0_model_pt86_lr2_b29_nqqp.txt', 'D:/habr_minitest_alt0_model_pt86_lr2_b29_nqqp.txt', traintarget, testtarget)
MiniCvScriptPasses('D:/habr_minitrain_alt0_model_pt87_lr2_b29_nqqp.txt', 'D:/habr_minitest_alt0_model_pt87_lr2_b29_nqqp.txt', traintarget, testtarget)
MiniCvScriptPasses('D:/habr_minitrain_alt0_model_pt88_lr2_b29_nqqp.txt', 'D:/habr_minitest_alt0_model_pt88_lr2_b29_nqqp.txt', traintarget, testtarget)
MiniCvScriptPasses('D:/habr_minitrain_alt0_model_pt89_lr2_b29_nqqp.txt', 'D:/habr_minitest_alt0_model_pt89_lr2_b29_nqqp.txt', traintarget, testtarget)
MiniCvScriptPasses('D:/habr_minitrain_alt0_model_pt9_lr2_b29_nqqp.txt', 'D:/habr_minitest_alt0_model_pt9_lr2_b29_nqqp.txt', traintarget, testtarget)


83999
0.384021979643
0.81179311308
83999
0.375972456541
0.810405860476
83999
0.36799914974
0.809379125191
83999
0.360142946273
0.808948381434
83999
0.352363608162
0.809298658982
83999
0.344741902023
0.810610361073


In [548]:
MiniCvScriptPasses('D:/habr_minitrain_alt0_model_pt88_lr15_b29_nqqp.txt', 'D:/habr_minitest_alt0_model_pt88_lr15_b29_nqqp.txt', traintarget, testtarget)
MiniCvScriptPasses('D:/habr_minitrain_alt0_model_pt88_lr17_b29_nqqp.txt', 'D:/habr_minitest_alt0_model_pt88_lr17_b29_nqqp.txt', traintarget, testtarget)
MiniCvScriptPasses('D:/habr_minitrain_alt0_model_pt88_lr2_b29_nqqp.txt', 'D:/habr_minitest_alt0_model_pt88_lr2_b29_nqqp.txt', traintarget, testtarget)
MiniCvScriptPasses('D:/habr_minitrain_alt0_model_pt88_lr23_b29_nqqp.txt', 'D:/habr_minitest_alt0_model_pt88_lr23_b29_nqqp.txt', traintarget, testtarget)
MiniCvScriptPasses('D:/habr_minitrain_alt0_model_pt88_lr25_b29_nqqp.txt', 'D:/habr_minitest_alt0_model_pt88_lr25_b29_nqqp.txt', traintarget, testtarget)


83999
0.369686249872
0.808724227438
83999
0.364820236158
0.808718967418
83999
0.360142946273
0.808948381434
83999
0.357521522471
0.809297356462
83999
0.356333367231
0.809509650399


In [555]:
MiniCvScriptPasses('D:/habr_minitrain_alt0up_model_pt88_lr17_b29_nqqp.txt', 'D:/habr_minitest_alt0up_model_pt88_lr17_b29_nqqp.txt', traintarget, testtarget)
MiniCvScriptPasses('D:/habr_minitrain_alt0_model_pt88_lr17_b29_nqqp1.txt', 'D:/habr_minitest_alt0_model_pt88_lr17_b29_nqqp1.txt', traintarget, testtarget)
MiniCvScriptPasses('D:/habr_minitrain_alt0_model_pt87_lr2_b29_nqqp1.txt', 'D:/habr_minitest_alt0_model_pt87_lr2_b29_nqqp1.txt', traintarget, testtarget)
MiniCvScriptPasses('D:/habr_minitrain_alt0_model_pt85_lr7_b29_nqqp1.txt', 'D:/habr_minitest_alt0_model_pt85_lr7_b29_nqqp1.txt', traintarget, testtarget)
MiniCvScriptPasses('D:/habr_minitrain_alt0_model_pt88_lr17_b29_nqqp2.txt', 'D:/habr_minitest_alt0_model_pt88_lr17_b29_nqqp2.txt', traintarget, testtarget)


83999
0.368077390064
0.818841733952
83999
0.313982939559
0.806266499764
83999
0.317085798914
0.806469582984
83999
0.361345037048
0.813365160079
83999
0.0917257155204
0.786536816227


In [581]:
MiniCvScriptPasses('D:/habr_minitrain_alt0_model_pt88_lr17_b29_nqq_t2.txt', 'D:/habr_minitest_alt0_model_pt88_lr17_b29_nqq_t2.txt', traintarget, testtarget)
MiniCvScriptPasses('D:/habr_minitrain_alt0_model_pt88_lr17_b29_nqq_t15.txt', 'D:/habr_minitest_alt0_model_pt88_lr17_b29_nqq_t15.txt', traintarget, testtarget)
MiniCvScriptPasses('D:/habr_minitrain_alt0_model_pt88_lr17_b29_nqq_t20.txt', 'D:/habr_minitest_alt0_model_pt88_lr17_b29_nqq_t20.txt', traintarget, testtarget)
MiniCvScriptPasses('D:/habr_minitrain_alt0_model_pt88_lr17_b29_nqq0_t20.txt', 'D:/habr_minitest_alt0_model_pt88_lr17_b29_nqq0_t20.txt', traintarget, testtarget)
MiniCvScriptPasses('D:/habr_minitrain_alt0_model_pt88_lr17_b29_nqq0_toff.txt', 'D:/habr_minitest_alt0_model_pt88_lr17_b29_nqq0_toff.txt', traintarget, testtarget, numpasses=29)


83999
0.168699552197
0.792080784812
83999
0.0681995687449
0.785037995056
83999
0.0543655211729
0.783838584067
83999
0.2875957677
0.80478316569
83999
17.670922413
17.7354987083


In [601]:
MiniCvScriptPasses('D:/habr_minitrain_alt0up2_model_pt88_lr17_b29_nqq.txt', 'D:/habr_minitest_alt0up2_model_pt88_lr17_b29_nqq.txt', traintarget, testtarget)
MiniCvScriptPasses('D:/habr_minitrain_alt0up3_model_pt88_lr17_b29_nqq.txt', 'D:/habr_minitest_alt0up3_model_pt88_lr17_b29_nqq.txt', traintarget, testtarget)
MiniCvScriptPasses('D:/habr_minitrain_alt0up4_model_pt88_lr17_b29_nqq.txt', 'D:/habr_minitest_alt0up4_model_pt88_lr17_b29_nqq.txt', traintarget, testtarget)


83999
0.309367966726
0.813485314713
83999
0.301542712532
0.817202551124
83999
0.306200179431
0.807636753118


In [607]:
MiniCvScriptPasses('D:/habr_minitrain_alt0_model_pt88_lr17_b29_nqq_t1.txt', 'D:/habr_minitest_alt0_model_pt88_lr17_b29_nqq_t1.txt', traintarget, testtarget, control = 0)
MiniCvScriptPasses('D:/habr_minitrain_alt0_model_pt88_lr17_b29_nqq_t2.txt', 'D:/habr_minitest_alt0_model_pt88_lr17_b29_nqq_t2.txt', traintarget, testtarget, control = 0)
MiniCvScriptPasses('D:/habr_minitrain_alt0_model_pt88_lr17_b29_nqq_t0.txt', 'D:/habr_minitest_alt0_model_pt88_lr17_b29_nqq_t0.txt', traintarget, testtarget, control = 0)
MiniCvScriptPasses('D:/habr_minitrain_alt0_model_pt88_lr17_b29_nqq_tt1.txt', 'D:/habr_minitest_alt0_model_pt88_lr17_b29_nqq_tt1.txt', traintarget, testtarget, control = 0)
MiniCvScriptPasses('D:/habr_minitrain_alt0_model_pt88_lr17_b29_nqq_tt2.txt', 'D:/habr_minitest_alt0_model_pt88_lr17_b29_nqq_tt2.txt', traintarget, testtarget, control = 0)
MiniCvScriptPasses('D:/habr_minitrain_alt0_model_pt88_lr17_b29_nqq_tt3.txt', 'D:/habr_minitest_alt0_model_pt88_lr17_b29_nqq_tt3.txt', traintarget, testtarget, control = 0)


83999
1.22006529409
0.806266499764
83999
0.298396295611
0.805207463959
83999
0.313982939559
0.806266499764
83999
0.270941007917
0.803726944529
83999
0.0600136390552
0.794941505958
83999
0.0973207783069
0.79763917269


(array([ 0.669951,  1.080408,  0.      , ...,  1.609439,  3.044522,
         4.110874]),
 array([ 0.597797,  0.009977,  1.431846, ...,  0.65376 ,  1.857215,
         2.410227]))

In [572]:
r1 = MiniCvScriptPasses('D:/habr_minitrain_alt0_model_pt88_lr17_b29_nqqp1.txt', 'D:/habr_minitest_alt0_model_pt88_lr17_b29_nqqp1.txt', traintarget, testtarget, control = 0)
r2 = MiniCvScriptPasses('D:/habr_minitrain_alt0_model_pt88_lr17_b29_nqqp2.txt', 'D:/habr_minitest_alt0_model_pt88_lr17_b29_nqqp2.txt', traintarget, testtarget, control = 0)
rt1 = (r1[0] + r2[0])/2
rt2 = (r1[1] + r2[1])/2
print(sklearn.metrics.mean_absolute_error(rt1, traintarget.target),
sklearn.metrics.mean_absolute_error(rt2, testtarget.target))


83999
0.313982939559
0.806266499764
83999
0.0917257155204
0.786536816227
0.199093616781 0.786625316977


In [ ]:
#pt 88, lr 17, b 29 --- 0.806266499764 instead of 0.809379125191, later triplets

In [518]:
len(train_target), len(traintarget)
traintargetnew = train_target[0:84000]
print(len(traintargetnew))
testtargetnew = train_target[84000:]
testtargetnew.reset_index(drop = True, inplace=True)

84000


In [532]:
#here we want to compare old models on a new train-test split
#first vw model
MiniCvScript('D:/habr_minitrain_altnew_model.txt', 'D:/habr_minitest_altnew_model.txt', traintargetnew, testtargetnew)
#second vw model pt = 0.85, lr = 0.7, b = 26
MiniCvScript('D:/habr_minitrain_altnew_model_pt85_lr7_b26.txt', 'D:/habr_minitest_altnew_model_pt85_lr7_b26.txt', traintargetnew, testtargetnew)
#third model quantile pt = 0.8, lr = 0.2, b = 29
MiniCvScript('D:/habr_minitrain_altnew_model_pt8_lr2_b29.txt', 'D:/habr_minitest_altnew_model_pt8_lr2_b29.txt', traintargetnew, testtargetnew)
#fourth model quantile, ngrams pt = 0.87, lr = 0.2, b =29
MiniCvScript('D:/habr_minitrain_altnew_model_pt87_lr2_b29.txt', 'D:/habr_minitest_altnew_model_pt87_lr2_b29.txt', traintargetnew, testtargetnew)
#passes
MiniCvScriptPasses('D:/habr_minitrain_altnew_model_pt87_lr2_b29_up.txt', 'D:/habr_minitest_altnew_model_pt87_lr2_b29_up.txt', traintargetnew, testtargetnew)
MiniCvScript('D:/habr_minitrain_altnew_model_pt5_lr5_b29.txt', 'D:/habr_minitest_altnew_model_pt5_lr5_b29.txt', traintargetnew, testtargetnew)


0.866371437298 0.943272809444
0.631022864988 0.946870442194
0.601879969798 0.962735238806
0.381382214571 0.941469307694
84000
0.3419669025
0.958929784167
0.718901341655 0.947662552222


In [531]:
MiniCvScript('D:/habr_minitrain_altnew_model_pt87_lr2_b29.txt', 'D:/habr_minitest_altnew_model_pt87_lr2_b29.txt', traintargetnew, testtargetnew)
MiniCvScript('D:/habr_minitrain_altnew_model_pt87_lr2_b24.txt', 'D:/habr_minitest_altnew_model_pt87_lr2_b24.txt', traintargetnew, testtargetnew)
MiniCvScript('D:/habr_minitrain_altnew_model_pt87_lr2_b18.txt', 'D:/habr_minitest_altnew_model_pt87_lr2_b18.txt', traintargetnew, testtargetnew)
MiniCvScript('D:/habr_minitrain_altnew_model_pt87_lr2_b12.txt', 'D:/habr_minitest_altnew_model_pt87_lr2_b12.txt', traintargetnew, testtargetnew)


0.381382214571 0.941469307694
0.383170366774 0.942482249
0.512139612726 0.985166792833
0.962871299202 1.10233468167


In [452]:
MiniCvScript('D:/habr_minitrain_alt2_model_pt87_lr2_b29_nqq.txt', 'D:/habr_minitest_alt2_model_pt87_lr2_b29_nqq.txt', traintarget, testtarget)
MiniCvScript('D:/habr_minitrain_alt2_model_pt87_lr2_b29_nqq3.txt', 'D:/habr_minitest_alt2_model_pt87_lr2_b29_nqq3.txt', traintarget, testtarget)
MiniCvScript('D:/habr_minitrain_alt2_model_pt87_lr2_b29_nqq4.txt', 'D:/habr_minitest_alt2_model_pt87_lr2_b29_nqq4.txt', traintarget, testtarget)


0.388281892201 0.836827084998
0.337460898904 0.83190611983
0.313149289134 0.833065322241


In [453]:
MiniCvScript('D:/habr_minitrain_alt2_model_pt8_lr2_b29_nqq3.txt', 'D:/habr_minitest_alt2_model_pt8_lr2_b29_nqq3.txt', traintarget, testtarget)
MiniCvScript('D:/habr_minitrain_alt2_model_pt85_lr2_b29_nqq3.txt', 'D:/habr_minitest_alt2_model_pt85_lr2_b29_nqq3.txt', traintarget, testtarget)
MiniCvScript('D:/habr_minitrain_alt2_model_pt87_lr2_b29_nqq3.txt', 'D:/habr_minitest_alt2_model_pt87_lr2_b29_nqq3.txt', traintarget, testtarget)
MiniCvScript('D:/habr_minitrain_alt2_model_pt9_lr2_b29_nqq3.txt', 'D:/habr_minitest_alt2_model_pt9_lr2_b29_nqq3.txt', traintarget, testtarget)


0.406636551376 0.850607261659
0.35502078945 0.835093622899
0.337460898904 0.83190611983
0.314617084108 0.832730367962


In [455]:
MiniCvScript('D:/habr_minitrain_alt2_model_pt87_lr1_b29_nqq3.txt', 'D:/habr_minitest_alt2_model_pt87_lr1_b29_nqq3.txt', traintarget, testtarget)
MiniCvScript('D:/habr_minitrain_alt2_model_pt87_lr2_b29_nqq3.txt', 'D:/habr_minitest_alt2_model_pt87_lr2_b29_nqq3.txt', traintarget, testtarget)
MiniCvScript('D:/habr_minitrain_alt2_model_pt87_lr3_b29_nqq3.txt', 'D:/habr_minitest_alt2_model_pt87_lr3_b29_nqq3.txt', traintarget, testtarget)
MiniCvScript('D:/habr_minitrain_alt2_model_pt87_lr4_b29_nqq3.txt', 'D:/habr_minitest_alt2_model_pt87_lr4_b29_nqq3.txt', traintarget, testtarget)
MiniCvScript('D:/habr_minitrain_alt2_model_pt87_lr5_b29_nqq3.txt', 'D:/habr_minitest_alt2_model_pt87_lr5_b29_nqq3.txt', traintarget, testtarget)
MiniCvScript('D:/habr_minitrain_alt2_model_pt87_lr6_b29_nqq3.txt', 'D:/habr_minitest_alt2_model_pt87_lr6_b29_nqq3.txt', traintarget, testtarget)
MiniCvScript('D:/habr_minitrain_alt2_model_pt87_lr7_b29_nqq3.txt', 'D:/habr_minitest_alt2_model_pt87_lr7_b29_nqq3.txt', traintarget, testtarget)
MiniCvScript('D:/habr_minitrain_alt2_model_pt87_lr8_b29_nqq3.txt', 'D:/habr_minitest_alt2_model_pt87_lr8_b29_nqq3.txt', traintarget, testtarget)
MiniCvScript('D:/habr_minitrain_alt2_model_pt87_lr9_b29_nqq3.txt', 'D:/habr_minitest_alt2_model_pt87_lr9_b29_nqq3.txt', traintarget, testtarget)


0.376240145799 0.836893533096
0.337460898904 0.83190611983
0.330595159085 0.830765064082
0.328909199764 0.830579950779
0.328807692484 0.836893533096
0.328529950916 0.836893533096
0.32776554003 0.836893533096
0.32781201175 0.836893533096
0.376240145799 0.836893533096


In [456]:
MiniCvScript('D:/habr_minitrain_alt2_model_pt8_lr4_b29_nqq3.txt', 'D:/habr_minitest_alt2_model_pt8_lr4_b29_nqq3.txt', traintarget, testtarget)
MiniCvScript('D:/habr_minitrain_alt2_model_pt85_lr4_b29_nqq3.txt', 'D:/habr_minitest_alt2_model_pt85_lr4_b29_nqq3.txt', traintarget, testtarget)
MiniCvScript('D:/habr_minitrain_alt2_model_pt87_lr4_b29_nqq3.txt', 'D:/habr_minitest_alt2_model_pt87_lr4_b29_nqq3.txt', traintarget, testtarget)
MiniCvScript('D:/habr_minitrain_alt2_model_pt9_lr4_b29_nqq3.txt', 'D:/habr_minitest_alt2_model_pt9_lr4_b29_nqq3.txt', traintarget, testtarget)


0.390589092549 0.848452083192
0.344782817593 0.833545930252
0.328909199764 0.830579950779
0.308290909987 0.831848099858


In [457]:
MiniCvScript('D:/habr_minitrain_alt2_model_pt87_lr4_b18_nqq3.txt', 'D:/habr_minitest_alt2_model_pt87_lr4_b18_nqq3.txt', traintarget, testtarget)


0.513174656924 0.943788091442


In [ ]:
#0.83057995 # 0.8319 vs. 0.83817220855

In [382]:
MiniCvScript('D:/habr_minirain_alt2_model_pt7_lr2_b26_q.txt', 'D:/habr_minitest_alt2_model_pt7_lr2_b26_q.txt', traintarget, testtarget)
MiniCvScript('D:/habr_minirain_alt2_model_pt8_lr2_b26_q.txt', 'D:/habr_minitest_alt2_model_pt8_lr2_b26_q.txt', traintarget, testtarget)
MiniCvScript('D:/habr_minirain_alt2_model_pt9_lr2_b26_q.txt', 'D:/habr_minitest_alt2_model_pt9_lr2_b26_q.txt', traintarget, testtarget)


0.746304901856 0.903138599594
0.655249997179 0.871599600983
0.589298245039 0.888542220827


In [423]:
MiniCvScript('D:/habr_minitrain_alt2_model_pt8_lr2_b26_nqq3.txt', 'D:/habr_minitest_alt2_model_pt8_lr2_b26_nqq3.txt', traintarget, testtarget)


0.407601768342 0.850985150468


In [391]:
with open('D:/habr_minitrain_alt2_model_pt85_lr7_b26.txt', 'r') as pred_file:
        valid_predictiontrain1 = [float(label) 
                             for label in pred_file.readlines()]
with open('D:/habr_minitest_alt2_model_pt85_lr7_b26.txt', 'r') as pred_file:
        valid_predictiontest1 = [float(label) 
                             for label in pred_file.readlines()]
with open('D:/habr_minirain_alt2_model_pt8_lr2_b26_q.txt', 'r') as pred_file:
        valid_predictiontrain2 = [float(label) 
                             for label in pred_file.readlines()]
with open('D:/habr_minitest_alt2_model_pt8_lr2_b26_q.txt', 'r') as pred_file:
        valid_predictiontest2 = [float(label) 
                             for label in pred_file.readlines()]
        
valid_predictiontrain3 = (np.array(valid_predictiontrain1) + np.array(valid_predictiontrain2))/2
valid_predictiontest3 = (np.array(valid_predictiontest1) + np.array(valid_predictiontest2))/2
print(sklearn.metrics.mean_absolute_error(valid_predictiontrain1, traintarget.target))
print(sklearn.metrics.mean_absolute_error(valid_predictiontrain2, traintarget.target))
print(sklearn.metrics.mean_absolute_error(valid_predictiontrain3, traintarget.target))
print(sklearn.metrics.mean_absolute_error(valid_predictiontest1, testtarget.target))
print(sklearn.metrics.mean_absolute_error(valid_predictiontest2, testtarget.target))
print(sklearn.metrics.mean_absolute_error(valid_predictiontest3, testtarget.target))

0.659552918213
0.655249997179
0.65325300806
0.887913585928
0.871599600983
0.874580421072


In [341]:
with open('D:/habr_train_alt2_model_pt9.txt', 'r') as pred_file:
        valid_predictiontrain = [float(label) 
                             for label in pred_file.readlines()]
with open('D:/habr_test_alt2_model_pt9.txt', 'r') as pred_file:
        valid_predictiontest = [float(label) 
                             for label in pred_file.readlines()]


In [342]:
sklearn.metrics.mean_absolute_error(valid_predictiontrain, train_target.target)

0.72229588502499986

In [343]:
your_submission3 = sample_sub.copy()
your_submission3['target'] = valid_predictiontest

In [346]:
your_submission3.describe(), your_submission.describe(), your_submissionbi.describe()

(             target
 count  52913.000000
 mean       2.402597
 std        1.185989
 min        0.000000
 25%        1.549127
 50%        2.370944
 75%        3.174151
 max        9.080687,              target
 count  52913.000000
 mean       2.310127
 std        1.089067
 min        0.000000
 25%        1.497322
 50%        2.262579
 75%        3.095786
 max        9.080687,              target
 count  52913.000000
 mean       2.310127
 std        1.089067
 min        0.000000
 25%        1.497322
 50%        2.262579
 75%        3.095786
 max        9.080687)

In [347]:
your_submission3.to_csv('D:/submission4.csv')

In [353]:
with open('D:/habr_train_alt2_model_pt85_lr55_b26.txt', 'r') as pred_file:
        valid_predictiontrain = [float(label) 
                             for label in pred_file.readlines()]
with open('D:/habr_test_alt2_model_pt85_lr55_b26.txt', 'r') as pred_file:
        valid_predictiontest = [float(label) 
                             for label in pred_file.readlines()]

In [354]:
sklearn.metrics.mean_absolute_error(valid_predictiontrain, train_target.target)

0.68677406215000003

In [355]:
your_submission4 = sample_sub.copy()
your_submission4['target'] = valid_predictiontest

In [356]:
your_submission4.describe(), your_submission3.describe(), your_submission.describe(), your_submissionbi.describe()

(             target
 count  52913.000000
 mean       2.396705
 std        1.116346
 min        0.000000
 25%        1.574642
 50%        2.390123
 75%        3.177296
 max        9.080687,              target
 count  52913.000000
 mean       2.402597
 std        1.185989
 min        0.000000
 25%        1.549127
 50%        2.370944
 75%        3.174151
 max        9.080687,              target
 count  52913.000000
 mean       2.310127
 std        1.089067
 min        0.000000
 25%        1.497322
 50%        2.262579
 75%        3.095786
 max        9.080687,              target
 count  52913.000000
 mean       2.310127
 std        1.089067
 min        0.000000
 25%        1.497322
 50%        2.262579
 75%        3.095786
 max        9.080687)

In [357]:
your_submission4.to_csv('D:/submission5.csv')

In [361]:
with open('D:/habr_train_alt2_model_pt85_lr55.txt', 'r') as pred_file:
        valid_predictiontrain = [float(label) 
                             for label in pred_file.readlines()]
with open('D:/habr_test_alt2_model_pt85_lr55.txt', 'r') as pred_file:
        valid_predictiontest = [float(label) 
                             for label in pred_file.readlines()]

In [362]:
sklearn.metrics.mean_absolute_error(valid_predictiontrain, train_target.target)

0.71756506365833339

In [363]:
your_submission5 = sample_sub.copy()
your_submission5['target'] = valid_predictiontest

In [364]:
your_submission5.describe(), your_submission4.describe(), your_submission3.describe(), your_submission.describe(), your_submissionbi.describe()

(             target
 count  52913.000000
 mean       2.398280
 std        1.121467
 min        0.000000
 25%        1.569046
 50%        2.393540
 75%        3.180255
 max        9.080687,              target
 count  52913.000000
 mean       2.396705
 std        1.116346
 min        0.000000
 25%        1.574642
 50%        2.390123
 75%        3.177296
 max        9.080687,              target
 count  52913.000000
 mean       2.402597
 std        1.185989
 min        0.000000
 25%        1.549127
 50%        2.370944
 75%        3.174151
 max        9.080687,              target
 count  52913.000000
 mean       2.310127
 std        1.089067
 min        0.000000
 25%        1.497322
 50%        2.262579
 75%        3.095786
 max        9.080687,              target
 count  52913.000000
 mean       2.310127
 std        1.089067
 min        0.000000
 25%        1.497322
 50%        2.262579
 75%        3.095786
 max        9.080687)

In [365]:
your_submission5.to_csv('D:/submission6.csv')

In [369]:
with open('D:/habr_train_alt2_model_pt85_lr7_b26.txt', 'r') as pred_file:
        valid_predictiontrain = [float(label) 
                             for label in pred_file.readlines()]
with open('D:/habr_test_alt2_model_pt85_lr7_b26.txt', 'r') as pred_file:
        valid_predictiontest = [float(label) 
                             for label in pred_file.readlines()]

In [370]:
sklearn.metrics.mean_absolute_error(valid_predictiontrain, train_target.target)

0.66510065227499993

In [371]:
your_submission6 = sample_sub.copy()
your_submission6['target'] = valid_predictiontest

In [372]:
your_submission6.describe(), your_submission5.describe(), your_submission4.describe(), your_submission3.describe(), your_submission.describe(), your_submissionbi.describe()

(             target
 count  52913.000000
 mean       2.400095
 std        1.133678
 min        0.000000
 25%        1.569993
 50%        2.388282
 75%        3.187404
 max        9.080687,              target
 count  52913.000000
 mean       2.398280
 std        1.121467
 min        0.000000
 25%        1.569046
 50%        2.393540
 75%        3.180255
 max        9.080687,              target
 count  52913.000000
 mean       2.396705
 std        1.116346
 min        0.000000
 25%        1.574642
 50%        2.390123
 75%        3.177296
 max        9.080687,              target
 count  52913.000000
 mean       2.402597
 std        1.185989
 min        0.000000
 25%        1.549127
 50%        2.370944
 75%        3.174151
 max        9.080687,              target
 count  52913.000000
 mean       2.310127
 std        1.089067
 min        0.000000
 25%        1.497322
 50%        2.262579
 75%        3.095786
 max        9.080687,              target
 count  52913.000000
 mean       2

In [373]:
your_submission6.to_csv('D:/submission7.csv')

In [383]:
with open('D:/habr_train_alt2_model_pt8_lr2_b26_q.txt', 'r') as pred_file:
        valid_predictiontrain = [float(label) 
                             for label in pred_file.readlines()]
with open('D:/habr_test_alt2_model_pt8_lr2_b26_q.txt', 'r') as pred_file:
        valid_predictiontest = [float(label) 
                             for label in pred_file.readlines()]

In [384]:
sklearn.metrics.mean_absolute_error(valid_predictiontrain, train_target.target)

0.66270571067499995

In [385]:
your_submission7 = sample_sub.copy()
your_submission7['target'] = valid_predictiontest

In [386]:
your_submission7.describe(), your_submission6.describe(), your_submission5.describe(), your_submission4.describe(), your_submission3.describe()

(             target
 count  52913.000000
 mean       2.401993
 std        1.152917
 min        0.000000
 25%        1.548429
 50%        2.445313
 75%        3.228436
 max        8.390436,              target
 count  52913.000000
 mean       2.400095
 std        1.133678
 min        0.000000
 25%        1.569993
 50%        2.388282
 75%        3.187404
 max        9.080687,              target
 count  52913.000000
 mean       2.398280
 std        1.121467
 min        0.000000
 25%        1.569046
 50%        2.393540
 75%        3.180255
 max        9.080687,              target
 count  52913.000000
 mean       2.396705
 std        1.116346
 min        0.000000
 25%        1.574642
 50%        2.390123
 75%        3.177296
 max        9.080687,              target
 count  52913.000000
 mean       2.402597
 std        1.185989
 min        0.000000
 25%        1.549127
 50%        2.370944
 75%        3.174151
 max        9.080687)

In [387]:
your_submission7.to_csv('D:/submission8.csv')

In [404]:
with open('D:/habr_train_alt2_model_pt8_lr4_b29_nqq.txt', 'r') as pred_file:
        valid_predictiontrain = [float(label) 
                             for label in pred_file.readlines()]
with open('D:/habr_test_alt2_model_pt8_lr4_b29_nqq.txt', 'r') as pred_file:
        valid_predictiontest = [float(label) 
                             for label in pred_file.readlines()]

In [405]:
sklearn.metrics.mean_absolute_error(valid_predictiontrain, train_target.target)

0.43911228925833329

In [406]:
your_submission8 = sample_sub.copy()
your_submission8['target'] = valid_predictiontest

In [408]:
your_submission8.describe(), your_submission7.describe(), your_submission6.describe(), your_submission5.describe(), your_submission4.describe(), your_submission3.describe()

(             target
 count  52913.000000
 mean       2.412789
 std        1.147967
 min        0.000000
 25%        1.556433
 50%        2.429637
 75%        3.238431
 max        8.022839,              target
 count  52913.000000
 mean       2.401993
 std        1.152917
 min        0.000000
 25%        1.548429
 50%        2.445313
 75%        3.228436
 max        8.390436,              target
 count  52913.000000
 mean       2.400095
 std        1.133678
 min        0.000000
 25%        1.569993
 50%        2.388282
 75%        3.187404
 max        9.080687,              target
 count  52913.000000
 mean       2.398280
 std        1.121467
 min        0.000000
 25%        1.569046
 50%        2.393540
 75%        3.180255
 max        9.080687,              target
 count  52913.000000
 mean       2.396705
 std        1.116346
 min        0.000000
 25%        1.574642
 50%        2.390123
 75%        3.177296
 max        9.080687,              target
 count  52913.000000
 mean       2

In [409]:
your_submission8.to_csv('D:/submission9.csv')

In [410]:
train_target.describe()

,target,antitarget
count,120000.000000,1.200000e+05
mean,2.828131,2.900000e+00
std,1.696940,2.649447e-12
min,0.000000,2.900000e+00
25%,1.609438,2.900000e+00
50%,2.944439,2.900000e+00
75%,4.127134,2.900000e+00
max,9.080687,2.900000e+00


In [412]:
with open('D:/habr_train_alt2_model_pt8_lr2_b26_nqq.txt', 'r') as pred_file:
        valid_predictiontrain = [float(label) 
                             for label in pred_file.readlines()]
with open('D:/habr_test_alt2_model_pt8_lr2_b26_nqq.txt', 'r') as pred_file:
        valid_predictiontest = [float(label) 
                             for label in pred_file.readlines()]

In [413]:
sklearn.metrics.mean_absolute_error(valid_predictiontrain, train_target.target)

0.47384692548333335

In [414]:
your_submission9 = sample_sub.copy()
your_submission9['target'] = valid_predictiontest

In [415]:
your_submission9.describe(), your_submission8.describe(), your_submission7.describe(), your_submission6.describe(), your_submission5.describe(), your_submission4.describe(), your_submission3.describe()

(             target
 count  52913.000000
 mean       2.415621
 std        1.131536
 min        0.000000
 25%        1.569363
 50%        2.448641
 75%        3.237877
 max        8.003355,              target
 count  52913.000000
 mean       2.412789
 std        1.147967
 min        0.000000
 25%        1.556433
 50%        2.429637
 75%        3.238431
 max        8.022839,              target
 count  52913.000000
 mean       2.401993
 std        1.152917
 min        0.000000
 25%        1.548429
 50%        2.445313
 75%        3.228436
 max        8.390436,              target
 count  52913.000000
 mean       2.400095
 std        1.133678
 min        0.000000
 25%        1.569993
 50%        2.388282
 75%        3.187404
 max        9.080687,              target
 count  52913.000000
 mean       2.398280
 std        1.121467
 min        0.000000
 25%        1.569046
 50%        2.393540
 75%        3.180255
 max        9.080687,              target
 count  52913.000000
 mean       2

In [416]:
your_submission9.to_csv('D:/submission10.csv')

In [417]:
with open('D:/habr_train_alt2_model_pt85_lr7_b26_nq.txt', 'r') as pred_file:
        valid_predictiontrain = [float(label) 
                             for label in pred_file.readlines()]
with open('D:/habr_test_alt2_model_pt85_lr7_b26_nq.txt', 'r') as pred_file:
        valid_predictiontest = [float(label) 
                             for label in pred_file.readlines()]

In [418]:
sklearn.metrics.mean_absolute_error(valid_predictiontrain, train_target.target)

0.50885518103333338

In [419]:
your_submission10 = sample_sub.copy()
your_submission10['target'] = valid_predictiontest

In [420]:
your_submission10.describe(), your_submission9.describe(), your_submission8.describe(), your_submission7.describe(), your_submission6.describe(), your_submission5.describe(), your_submission4.describe(), your_submission3.describe()

(             target
 count  52913.000000
 mean       2.416969
 std        1.093079
 min        0.000000
 25%        1.600574
 50%        2.409858
 75%        3.194322
 max        8.733557,              target
 count  52913.000000
 mean       2.415621
 std        1.131536
 min        0.000000
 25%        1.569363
 50%        2.448641
 75%        3.237877
 max        8.003355,              target
 count  52913.000000
 mean       2.412789
 std        1.147967
 min        0.000000
 25%        1.556433
 50%        2.429637
 75%        3.238431
 max        8.022839,              target
 count  52913.000000
 mean       2.401993
 std        1.152917
 min        0.000000
 25%        1.548429
 50%        2.445313
 75%        3.228436
 max        8.390436,              target
 count  52913.000000
 mean       2.400095
 std        1.133678
 min        0.000000
 25%        1.569993
 50%        2.388282
 75%        3.187404
 max        9.080687,              target
 count  52913.000000
 mean       2

In [421]:
your_submission10.to_csv('D:/submission11.csv')

In [441]:
with open('D:/habr_train_alt2_model_pt85_lr3_b29_nqq.txt', 'r') as pred_file:
        valid_predictiontrain = [float(label) 
                             for label in pred_file.readlines()]
with open('D:/habr_test_alt2_model_pt85_lr3_b29_nqq.txt', 'r') as pred_file:
        valid_predictiontest = [float(label) 
                             for label in pred_file.readlines()]

In [442]:
sklearn.metrics.mean_absolute_error(valid_predictiontrain, train_target.target)

0.399207711425

In [443]:
your_submission11 = sample_sub.copy()
your_submission11['target'] = valid_predictiontest

In [444]:
your_submission11.describe(), your_submission10.describe(), your_submission9.describe(), your_submission8.describe(), your_submission7.describe(), your_submission6.describe(), your_submission5.describe(), your_submission4.describe(), your_submission3.describe()

(             target
 count  52913.000000
 mean       2.434291
 std        1.170792
 min        0.000000
 25%        1.571639
 50%        2.451507
 75%        3.260686
 max        8.825754,              target
 count  52913.000000
 mean       2.416969
 std        1.093079
 min        0.000000
 25%        1.600574
 50%        2.409858
 75%        3.194322
 max        8.733557,              target
 count  52913.000000
 mean       2.415621
 std        1.131536
 min        0.000000
 25%        1.569363
 50%        2.448641
 75%        3.237877
 max        8.003355,              target
 count  52913.000000
 mean       2.412789
 std        1.147967
 min        0.000000
 25%        1.556433
 50%        2.429637
 75%        3.238431
 max        8.022839,              target
 count  52913.000000
 mean       2.401993
 std        1.152917
 min        0.000000
 25%        1.548429
 50%        2.445313
 75%        3.228436
 max        8.390436,              target
 count  52913.000000
 mean       2

In [445]:
your_submission11.to_csv('D:/submission12.csv')

In [458]:
with open('D:/habr_train_alt2_model_pt87_lr4_b29_nqq3.txt', 'r') as pred_file:
        valid_predictiontrain = [float(label) 
                             for label in pred_file.readlines()]
with open('D:/habr_test_alt2_model_pt87_lr4_b29_nqq3.txt', 'r') as pred_file:
        valid_predictiontest = [float(label) 
                             for label in pred_file.readlines()]

In [459]:
sklearn.metrics.mean_absolute_error(valid_predictiontrain, train_target.target)


0.33392182323333341

In [460]:
your_submission12 = sample_sub.copy()
your_submission12['target'] = valid_predictiontest

In [461]:
your_submission12.describe(), your_submission11.describe(), your_submission10.describe(), your_submission9.describe(), your_submission8.describe(), your_submission7.describe(), your_submission6.describe(), your_submission5.describe(), your_submission4.describe(), your_submission3.describe()

(             target
 count  52913.000000
 mean       2.482600
 std        1.141198
 min        0.000000
 25%        1.662531
 50%        2.511976
 75%        3.279460
 max        9.080687,              target
 count  52913.000000
 mean       2.434291
 std        1.170792
 min        0.000000
 25%        1.571639
 50%        2.451507
 75%        3.260686
 max        8.825754,              target
 count  52913.000000
 mean       2.416969
 std        1.093079
 min        0.000000
 25%        1.600574
 50%        2.409858
 75%        3.194322
 max        8.733557,              target
 count  52913.000000
 mean       2.415621
 std        1.131536
 min        0.000000
 25%        1.569363
 50%        2.448641
 75%        3.237877
 max        8.003355,              target
 count  52913.000000
 mean       2.412789
 std        1.147967
 min        0.000000
 25%        1.556433
 50%        2.429637
 75%        3.238431
 max        8.022839,              target
 count  52913.000000
 mean       2

In [462]:
your_submission12.to_csv('D:/submission13.csv')

In [155]:
def MiniCvScriptPassesWar(trainfile, testfile, traintab, numpasses = 4):
    with open(trainfile, 'r') as pred_file:
        valid_predictiontrain = [float(label) 
                             for label in pred_file.readlines()]
    with open(testfile, 'r') as pred_file:
        valid_predictiontest = [float(label) 
                             for label in pred_file.readlines()]
    valid_predictiontrain = np.array(valid_predictiontrain)
    valid_predictiontest = np.array(valid_predictiontest)
    #mlen = int(len(valid_predictiontrain)/numpasses)
    #trainarr = valid_predictiontrain[0:mlen]
    #testarr = valid_predictiontest[0:mlen]
    #for j in range(1,numpasses):
    #    trainarr += valid_predictiontrain[j*mlen:(j+1)*mlen]
    #trainarr = trainarr/4
    #mlen = int(len(valid_predictiontest)/numpasses)    
    #testarr = valid_predictiontest[0:mlen]
    #for j in range(1,numpasses):
    #    testarr += valid_predictiontest[j*mlen:(j+1)*mlen]
    #testarr = testarr/4

    trainarr = valid_predictiontrain
    testarr = valid_predictiontest
    
    print(len(trainarr))
    print(sklearn.metrics.mean_absolute_error(trainarr, traintab.target))
    return(testarr)
    #print(sklearn.metrics.mean_absolute_error(valid_predictiontrain, traintab.target), sklearn.metrics.mean_absolute_error(valid_predictiontest, testtab.target))    

In [524]:
newpred = MiniCvScriptPassesWar('D:/habr_train_alt_model_pt87_lr2_b29_up.txt', 'D:/habr_test_alt_model_pt87_lr2_b29_up.txt', train_target)

120000
0.366588698617


In [528]:
your_submission13 = sample_sub.copy()
your_submission13['target'] = newpred


In [530]:
your_submission13.describe(), your_submission12.describe()

(             target
 count  52913.000000
 mean       2.523937
 std        1.196939
 min        0.000000
 25%        1.668195
 50%        2.556087
 75%        3.352897
 max        9.080687,              target
 count  52913.000000
 mean       2.482600
 std        1.141198
 min        0.000000
 25%        1.662531
 50%        2.511976
 75%        3.279460
 max        9.080687)

In [536]:
your_submission13.to_csv('D:/submission14.csv')

In [541]:
newpred = MiniCvScriptPassesWar('D:/habr_train_alt0_model_pt87_lr2_b29_nqqp.txt', 'D:/habr_test_alt0_model_pt87_lr2_b29_nqqp.txt', train_target)

120000
0.379550111383


In [542]:
your_submission14 = sample_sub.copy()
your_submission14['target'] = newpred
len(newpred)

52913

In [543]:
your_submission14.to_csv('D:/submission15.csv')

In [60]:
newpred = MiniCvScriptPassesWar('D:/habr_train_alt0_model_pt88_lr17_b29_nqqp1.txt', 'D:/habr_test_alt0_model_pt88_lr17_b29_nqqp1.txt', train_target)

120000
0.325696955442


In [61]:
your_submission15 = sample_sub.copy()
your_submission15['target'] = newpred
len(newpred)

52913

In [560]:
your_submission14.describe(), your_submission15.describe()

(             target
 count  52913.000000
 mean       2.511292
 std        1.185394
 min        0.000000
 25%        1.658423
 50%        2.548477
 75%        3.342152
 max        8.932331,              target
 count  52913.000000
 mean       2.519580
 std        1.193069
 min        0.000000
 25%        1.663498
 50%        2.553678
 75%        3.352393
 max        9.080687)

In [561]:
your_submission15.to_csv('D:/submission16.csv')

In [562]:
newpred = MiniCvScriptPassesWar('D:/habr_train_alt0_model_pt88_lr17_b29_nqqp2.txt', 'D:/habr_test_alt0_model_pt88_lr17_b29_nqqp2.txt', train_target)

120000
0.0948860983583


In [563]:
your_submission16 = sample_sub.copy()
your_submission16['target'] = newpred
len(newpred)

52913

In [564]:
your_submission14.describe(), your_submission15.describe(), your_submission16.describe()

(             target
 count  52913.000000
 mean       2.511292
 std        1.185394
 min        0.000000
 25%        1.658423
 50%        2.548477
 75%        3.342152
 max        8.932331,              target
 count  52913.000000
 mean       2.519580
 std        1.193069
 min        0.000000
 25%        1.663498
 50%        2.553678
 75%        3.352393
 max        9.080687,              target
 count  52913.000000
 mean       2.714724
 std        1.156414
 min        0.000000
 25%        1.918919
 50%        2.765654
 75%        3.518524
 max        8.976396)

In [565]:
your_submission16.to_csv('D:/submission17.csv')

In [573]:
your_submission17 = sample_sub.copy()
your_submission17['target'] = (your_submission16.target + your_submission15.target)/2

In [574]:
your_submission17.describe(), your_submission16.describe(), your_submission15.describe()

(             target
 count  52913.000000
 mean       2.617152
 std        1.171663
 min        0.000000
 25%        1.790750
 50%        2.659937
 75%        3.433495
 max        9.021225,              target
 count  52913.000000
 mean       2.714724
 std        1.156414
 min        0.000000
 25%        1.918919
 50%        2.765654
 75%        3.518524
 max        8.976396,              target
 count  52913.000000
 mean       2.519580
 std        1.193069
 min        0.000000
 25%        1.663498
 50%        2.553678
 75%        3.352393
 max        9.080687)

In [575]:
your_submission17.to_csv('D:/submission18.csv')

In [589]:
your_submission18 = sample_sub.copy()
your_submission18['target'] = (0.525*your_submission16.target + 0.475*your_submission15.target)

In [590]:
your_submission18.describe(), your_submission17.describe(), your_submission16.describe(), your_submission15.describe()

(             target
 count  52913.000000
 mean       2.622031
 std        1.170752
 min        0.000000
 25%        1.797972
 50%        2.665186
 75%        3.437979
 max        9.018984,              target
 count  52913.000000
 mean       2.617152
 std        1.171663
 min        0.000000
 25%        1.790750
 50%        2.659937
 75%        3.433495
 max        9.021225,              target
 count  52913.000000
 mean       2.714724
 std        1.156414
 min        0.000000
 25%        1.918919
 50%        2.765654
 75%        3.518524
 max        8.976396,              target
 count  52913.000000
 mean       2.519580
 std        1.193069
 min        0.000000
 25%        1.663498
 50%        2.553678
 75%        3.352393
 max        9.080687)

In [591]:
your_submission18.to_csv('D:/submission19.csv')

In [608]:
newpred1 = MiniCvScriptPassesWar('D:/habr_train_alt0_model_pt88_lr17_b29_nqq_sf1.txt', 'D:/habr_test_alt0_model_pt88_lr17_b29_nqq_sf1.txt', train_target)
newpred2 = MiniCvScriptPassesWar('D:/habr_train_alt0_model_pt88_lr17_b29_nqq_sf2.txt', 'D:/habr_test_alt0_model_pt88_lr17_b29_nqq_sf2.txt', train_target)


120000
0.310893552683
120000
0.093649326675


In [610]:
your_submission19 = sample_sub.copy()
your_submission19['target'] = (np.array(newpred1) + np.array(newpred2))/2

In [611]:
your_submission19.describe()

,target
count,52913.000000
mean,2.626787
std,1.173781
min,0.000000
25%,1.799338
50%,2.678384
75%,3.446311
max,8.911058


In [612]:
your_submission19.to_csv('D:/submission20.csv')

In [613]:
newpred1 = MiniCvScriptPassesWar('D:/habr_train_alt0_model_pt88_lr17_b29_nqq_h202.txt', 'D:/habr_test_alt0_model_pt88_lr17_b29_nqq_h202.txt', train_target)
newpred2 = MiniCvScriptPassesWar('D:/habr_train_alt0_model_pt88_lr17_b29_nqq_h203.txt', 'D:/habr_test_alt0_model_pt88_lr17_b29_nqq_h203.txt', train_target)


120000
0.302982759058
120000
0.0583795785833


In [614]:
your_submission20 = sample_sub.copy()
your_submission20['target'] = (np.array(newpred1) + np.array(newpred2))/2

In [615]:
your_submission20.describe()

,target
count,52913.000000
mean,2.622149
std,1.177189
min,0.000000
25%,1.788918
50%,2.669519
75%,3.446921
max,9.003546


In [616]:
your_submission20.to_csv('D:/submission21.csv')

In [617]:
newpred1 = MiniCvScriptPassesWar('D:/habr_train_alt0_model_pt88_lr2_b29_nqq.txt', 'D:/habr_test_alt0_model_pt88_lr2_b29_nqq.txt', train_target)
newpred2 = MiniCvScriptPassesWar('D:/habr_train_alt0_model_pt88_lr2_b29_nqq2.txt', 'D:/habr_test_alt0_model_pt88_lr2_b29_nqq2.txt', train_target)


120000
0.319404217433
120000
0.0946544323167


In [618]:
your_submission21 = sample_sub.copy()
your_submission21['target'] = (np.array(newpred1) + np.array(newpred2))/2

In [619]:
your_submission21.describe()

,target
count,52913.000000
mean,2.616157
std,1.174122
min,0.000000
25%,1.786822
50%,2.658640
75%,3.435538
max,9.021659


In [620]:
your_submission21.to_csv('D:/submission22.csv')

In [622]:
newpred1 = MiniCvScriptPassesWar('D:/habr_train_alt0_model_pt85_lr17_b29_nqq.txt', 'D:/habr_test_alt0_model_pt85_lr17_b29_nqq.txt', train_target)
newpred2 = MiniCvScriptPassesWar('D:/habr_train_alt0_model_pt85_lr17_b29_nqq2.txt', 'D:/habr_test_alt0_model_pt85_lr17_b29_nqq2.txt', train_target)


120000
0.353234450333
120000
0.300611807267


In [623]:
your_submission22 = sample_sub.copy()
your_submission22['target'] = (np.array(newpred1) + np.array(newpred2))/2

In [624]:
your_submission22.describe()

,target
count,52913.000000
mean,2.527730
std,1.150397
min,0.000000
25%,1.708311
50%,2.583778
75%,3.350817
max,8.244503


In [625]:
your_submission22.to_csv('D:/submission23.csv')

In [85]:
newpred1 = MiniCvScriptPassesWar('D:/habr_train_alt0_model_pt91_lr17_b29_nqq.txt', 'D:/habr_test_alt0_model_pt91_lr17_b29_nqq.txt', train_target)
newpred2 = MiniCvScriptPassesWar('D:/habr_train_alt0_model_pt91_lr17_b29_nqq2.txt', 'D:/habr_test_alt0_model_pt91_lr17_b29_nqq2.txt', train_target)


120000
0.298599071233
120000
0.0934039205167


In [86]:
your_submission23 = sample_sub.copy()
your_submission23['target'] = (np.array(newpred1) + np.array(newpred2))/2

In [87]:
your_submission23.describe()

,target
count,52913.000000
mean,2.622841
std,1.190220
min,0.000000
25%,1.790247
50%,2.643789
75%,3.422678
max,9.080687


In [629]:
your_submission23.to_csv('D:/submission24.csv')

In [646]:
newpred1 = MiniCvScriptPassesWar('D:/habr_train_alt0_model_pt91_lr14_b29_nqq1.txt', 'D:/habr_test_alt0_model_pt91_lr14_b29_nqq1.txt', train_target)
newpred2 = MiniCvScriptPassesWar('D:/habr_train_alt0_model_pt91_lr14_b29_nqq2.txt', 'D:/habr_test_alt0_model_pt91_lr14_b29_nqq2.txt', train_target)


120000
0.305842408167
120000
0.0938269930083


In [647]:
your_submission24 = sample_sub.copy()
your_submission24['target'] = (np.array(newpred1) + np.array(newpred2))/2

In [648]:
your_submission24.describe()

,target
count,52913.000000
mean,2.624733
std,1.187155
min,0.000000
25%,1.794069
50%,2.645288
75%,3.421878
max,9.080687


In [649]:
your_submission24.to_csv('D:/submission25.csv')

In [653]:
newpred1 = MiniCvScriptPassesWar('D:/habr_train_alt0_model_pt94_lr17_b29_nqq1.txt', 'D:/habr_test_alt0_model_pt94_lr17_b29_nqq1.txt', train_target)
newpred2 = MiniCvScriptPassesWar('D:/habr_train_alt0_model_pt94_lr17_b29_nqq2.txt', 'D:/habr_test_alt0_model_pt94_lr17_b29_nqq2.txt', train_target)


120000
0.273460814267
120000
0.0965939985


In [654]:
your_submission25 = sample_sub.copy()
your_submission25['target'] = (np.array(newpred1) + np.array(newpred2))/2

In [655]:
your_submission25.describe()

,target
count,52913.000000
mean,2.627801
std,1.227498
min,0.000000
25%,1.779082
50%,2.611610
75%,3.410500
max,9.080687


In [656]:
len(your_submission25)

52913

In [657]:
your_submission25.to_csv('D:/submission26.csv')

In [6]:
mytrain = pd.DataFrame(traintargetan, columns = ['target'])
mytest = pd.DataFrame(testtargetan, columns = ['target'])


In [11]:
#test1 what is better square or quantile
import sklearn
MiniCvScriptPasses('D:/habr_minitrain_alt0_an_model_pt5_lr5_b29_new1.txt', 'D:/habr_minitest_alt0_an_model_pt5_lr5_b29_new1.txt', mytrain, mytest)
MiniCvScriptPasses('D:/habr_minitrain_alt0_an_model_pt5_lr5_b29_new2.txt', 'D:/habr_minitest_alt0_an_model_pt5_lr5_b29_new2.txt', mytrain, mytest)
#so squared is a bit better.
#test2 with sort features or not
MiniCvScriptPasses('D:/habr_minitrain_alt0_an_model_pt5_lr5_b29_new3.txt', 'D:/habr_minitest_alt0_an_model_pt5_lr5_b29_new3.txt', mytrain, mytest)
MiniCvScriptPasses('D:/habr_minitrain_alt0_an_model_pt5_lr5_b29_new4.txt', 'D:/habr_minitest_alt0_an_model_pt5_lr5_b29_new4.txt', mytrain, mytest)
#mean, without sort features

84000
0.522228474393
0.930222871917
84000
0.62247904794
0.935949071139
84000
0.5098774975
0.93147848175
84000
0.61482999256
0.935204435639


In [12]:
#choice of optimal learning rate
MiniCvScriptPasses('D:/habr_minitrain_alt0_an_model_pt5_lr1_b29_new1.txt', 'D:/habr_minitest_alt0_an_model_pt5_lr1_b29_new1.txt', mytrain, mytest)
MiniCvScriptPasses('D:/habr_minitrain_alt0_an_model_pt5_lr2_b29_new1.txt', 'D:/habr_minitest_alt0_an_model_pt5_lr2_b29_new1.txt', mytrain, mytest)
MiniCvScriptPasses('D:/habr_minitrain_alt0_an_model_pt5_lr3_b29_new1.txt', 'D:/habr_minitest_alt0_an_model_pt5_lr3_b29_new1.txt', mytrain, mytest)
MiniCvScriptPasses('D:/habr_minitrain_alt0_an_model_pt5_lr4_b29_new1.txt', 'D:/habr_minitest_alt0_an_model_pt5_lr4_b29_new1.txt', mytrain, mytest)
MiniCvScriptPasses('D:/habr_minitrain_alt0_an_model_pt5_lr5_b29_new1.txt', 'D:/habr_minitest_alt0_an_model_pt5_lr5_b29_new1.txt', mytrain, mytest)
MiniCvScriptPasses('D:/habr_minitrain_alt0_an_model_pt5_lr6_b29_new1.txt', 'D:/habr_minitest_alt0_an_model_pt5_lr6_b29_new1.txt', mytrain, mytest)
MiniCvScriptPasses('D:/habr_minitrain_alt0_an_model_pt5_lr7_b29_new1.txt', 'D:/habr_minitest_alt0_an_model_pt5_lr7_b29_new1.txt', mytrain, mytest)
MiniCvScriptPasses('D:/habr_minitrain_alt0_an_model_pt5_lr8_b29_new1.txt', 'D:/habr_minitest_alt0_an_model_pt5_lr8_b29_new1.txt', mytrain, mytest)
MiniCvScriptPasses('D:/habr_minitrain_alt0_an_model_pt5_lr9_b29_new1.txt', 'D:/habr_minitest_alt0_an_model_pt5_lr9_b29_new1.txt', mytrain, mytest)
###lr = 0.9 ????

84000
0.352515304464
0.97425099125
84000
0.440319663202
0.946911778389
84000
0.476383863524
0.938655803889
84000
0.528465177976
0.932590812583
84000
0.522228474393
0.930222871917
84000
0.615357335357
0.926505578639
84000
0.612159711
0.925752185028
84000
0.609814703286
0.925456511806
84000
0.608106213583
0.92538208975


In [13]:
#choice of power_t
MiniCvScriptPasses('D:/habr_minitrain_alt0_an_model_pt1_lr9_b29_new1.txt', 'D:/habr_minitest_alt0_an_model_pt1_lr9_b29_new1.txt', mytrain, mytest)
MiniCvScriptPasses('D:/habr_minitrain_alt0_an_model_pt2_lr9_b29_new1.txt', 'D:/habr_minitest_alt0_an_model_pt2_lr9_b29_new1.txt', mytrain, mytest)
MiniCvScriptPasses('D:/habr_minitrain_alt0_an_model_pt3_lr9_b29_new1.txt', 'D:/habr_minitest_alt0_an_model_pt3_lr9_b29_new1.txt', mytrain, mytest)
MiniCvScriptPasses('D:/habr_minitrain_alt0_an_model_pt4_lr9_b29_new1.txt', 'D:/habr_minitest_alt0_an_model_pt4_lr9_b29_new1.txt', mytrain, mytest)
MiniCvScriptPasses('D:/habr_minitrain_alt0_an_model_pt5_lr9_b29_new1.txt', 'D:/habr_minitest_alt0_an_model_pt5_lr9_b29_new1.txt', mytrain, mytest)
MiniCvScriptPasses('D:/habr_minitrain_alt0_an_model_pt6_lr9_b29_new1.txt', 'D:/habr_minitest_alt0_an_model_pt6_lr9_b29_new1.txt', mytrain, mytest)
MiniCvScriptPasses('D:/habr_minitrain_alt0_an_model_pt7_lr9_b29_new1.txt', 'D:/habr_minitest_alt0_an_model_pt7_lr9_b29_new1.txt', mytrain, mytest)
MiniCvScriptPasses('D:/habr_minitrain_alt0_an_model_pt8_lr9_b29_new1.txt', 'D:/habr_minitest_alt0_an_model_pt8_lr9_b29_new1.txt', mytrain, mytest)
MiniCvScriptPasses('D:/habr_minitrain_alt0_an_model_pt9_lr9_b29_new1.txt', 'D:/habr_minitest_alt0_an_model_pt9_lr9_b29_new1.txt', mytrain, mytest)


84000
0.846375969238
0.9186078785
84000
0.842766383976
0.915187498056
84000
0.803093321536
0.918302270861
84000
0.759949587
0.921063984222
84000
0.608106213583
0.92538208975
84000
0.47591757531
0.933853586722
84000
0.392026715548
0.943426791806
84000
0.342075988095
0.949605893083
84000
0.286373409393
0.945371380306


In [15]:
MiniCvScriptPasses('D:/habr_minitrain_alt0_an_model_pt2_lr1_b29_new1.txt', 'D:/habr_minitest_alt0_an_model_pt2_lr1_b29_new1.txt', mytrain, mytest)
MiniCvScriptPasses('D:/habr_minitrain_alt0_an_model_pt2_lr2_b29_new1.txt', 'D:/habr_minitest_alt0_an_model_pt2_lr2_b29_new1.txt', mytrain, mytest)
MiniCvScriptPasses('D:/habr_minitrain_alt0_an_model_pt2_lr3_b29_new1.txt', 'D:/habr_minitest_alt0_an_model_pt2_lr3_b29_new1.txt', mytrain, mytest)
MiniCvScriptPasses('D:/habr_minitrain_alt0_an_model_pt2_lr4_b29_new1.txt', 'D:/habr_minitest_alt0_an_model_pt2_lr4_b29_new1.txt', mytrain, mytest)
MiniCvScriptPasses('D:/habr_minitrain_alt0_an_model_pt2_lr5_b29_new1.txt', 'D:/habr_minitest_alt0_an_model_pt2_lr5_b29_new1.txt', mytrain, mytest)
MiniCvScriptPasses('D:/habr_minitrain_alt0_an_model_pt2_lr6_b29_new1.txt', 'D:/habr_minitest_alt0_an_model_pt2_lr6_b29_new1.txt', mytrain, mytest)
MiniCvScriptPasses('D:/habr_minitrain_alt0_an_model_pt2_lr7_b29_new1.txt', 'D:/habr_minitest_alt0_an_model_pt2_lr7_b29_new1.txt', mytrain, mytest)
MiniCvScriptPasses('D:/habr_minitrain_alt0_an_model_pt2_lr8_b29_new1.txt', 'D:/habr_minitest_alt0_an_model_pt2_lr8_b29_new1.txt', mytrain, mytest)
MiniCvScriptPasses('D:/habr_minitrain_alt0_an_model_pt2_lr9_b29_new1.txt', 'D:/habr_minitest_alt0_an_model_pt2_lr9_b29_new1.txt', mytrain, mytest)
MiniCvScriptPasses('D:/habr_minitrain_alt0_an_model_pt2_lr95_b29_new1.txt', 'D:/habr_minitest_alt0_an_model_pt2_lr95_b29_new1.txt', mytrain, mytest)


84000
0.68081970244
0.918325969417
84000
0.816303984833
0.916730162944
84000
0.838843251929
0.915910620056
84000
0.856661355905
0.915531984667
84000
0.846709911952
0.915035787278
84000
0.86136563344
0.915163702556
84000
0.853342850952
0.915037889722
84000
0.847296784286
0.915082816583
84000
0.842766383976
0.915187498056
84000
0.840985641917
0.915259803


In [16]:
MiniCvScriptPasses('D:/habr_minitrain_alt0_an_model_pt85_lr4_b29_new1.txt', 'D:/habr_minitest_alt0_an_model_pt85_lr4_b29_new1.txt', mytrain, mytest)


84000
0.120850829738
0.997276805167


In [17]:
#OK power t = 0.2, lr = 0.5
MiniCvScriptPasses('D:/habr_minitrain_alt0_an_model_pt1_lr5_b29_new1.txt', 'D:/habr_minitest_alt0_an_model_pt1_lr5_b29_new1.txt', mytrain, mytest)
MiniCvScriptPasses('D:/habr_minitrain_alt0_an_model_pt2_lr5_b29_new1.txt', 'D:/habr_minitest_alt0_an_model_pt2_lr5_b29_new1.txt', mytrain, mytest)
MiniCvScriptPasses('D:/habr_minitrain_alt0_an_model_pt3_lr5_b29_new1.txt', 'D:/habr_minitest_alt0_an_model_pt3_lr5_b29_new1.txt', mytrain, mytest)
MiniCvScriptPasses('D:/habr_minitrain_alt0_an_model_pt4_lr5_b29_new1.txt', 'D:/habr_minitest_alt0_an_model_pt4_lr5_b29_new1.txt', mytrain, mytest)
MiniCvScriptPasses('D:/habr_minitrain_alt0_an_model_pt5_lr5_b29_new1.txt', 'D:/habr_minitest_alt0_an_model_pt5_lr5_b29_new1.txt', mytrain, mytest)
MiniCvScriptPasses('D:/habr_minitrain_alt0_an_model_pt6_lr5_b29_new1.txt', 'D:/habr_minitest_alt0_an_model_pt6_lr5_b29_new1.txt', mytrain, mytest)
MiniCvScriptPasses('D:/habr_minitrain_alt0_an_model_pt7_lr5_b29_new1.txt', 'D:/habr_minitest_alt0_an_model_pt7_lr5_b29_new1.txt', mytrain, mytest)
MiniCvScriptPasses('D:/habr_minitrain_alt0_an_model_pt8_lr5_b29_new1.txt', 'D:/habr_minitest_alt0_an_model_pt8_lr5_b29_new1.txt', mytrain, mytest)
MiniCvScriptPasses('D:/habr_minitrain_alt0_an_model_pt9_lr5_b29_new1.txt', 'D:/habr_minitest_alt0_an_model_pt9_lr5_b29_new1.txt', mytrain, mytest)


84000
0.862476740571
0.915525298111
84000
0.846709911952
0.915035787278
84000
0.810933663214
0.918166028389
84000
0.664877559952
0.921696985639
84000
0.522228474393
0.930222871917
84000
0.412482799214
0.943043677778
84000
0.362989328107
0.955505727194
84000
0.360250229083
0.963010765611
84000
0.122167353857
0.949605893083


In [30]:
#with trigrams
MiniCvScriptPasses('D:/habr_minitrain_alt0_an_model_pt2_lr5_b29_new11.txt', 'D:/habr_minitest_alt0_an_model_pt2_lr5_b29_new11.txt', mytrain, mytest)


84000
0.833226438202
0.919477778083


In [20]:
newpred = MiniCvScriptPassesWar('D:/habr_train_alt0_model_pt2_lr5_b29_new1.txt', 'D:/habr_test_alt0_model_pt2_lr5_b29_new1.txt', train_target)


120000
0.623915588317


In [23]:
your_submission26 = sample_sub.copy()
your_submission26['target'] = newpred

In [29]:
your_submission26.describe()

,target
count,52913.000000
mean,2.526300
std,1.131510
min,0.000000
25%,1.695763
50%,2.481793
75%,3.331515
max,8.256542


In [25]:
newpred = MiniCvScriptPassesWar('D:/habr_train_alt0_model_pt5_lr5_b29_new1.txt', 'D:/habr_test_alt0_model_pt5_lr5_b29_new1.txt', train_target)


120000
0.568141554925


In [26]:
your_submission27 = sample_sub.copy()
your_submission27['target'] = newpred

In [27]:
your_submission27.describe()

,target
count,52913.000000
mean,2.346060
std,1.061850
min,0.000000
25%,1.555201
50%,2.323987
75%,3.112196
max,7.622062


In [28]:
your_submission26.to_csv('D:/submission27.csv')

In [36]:
your_submission27.to_csv('D:/submission28.csv')

In [32]:
#OK, suppose we have quantile with sort features...
MiniCvScriptPasses('D:/habr_minitrain_alt0_an_model_pt5_lr1_b29_new4.txt', 'D:/habr_minitest_alt0_an_model_pt5_lr1_b29_new4.txt', mytrain, mytest)
MiniCvScriptPasses('D:/habr_minitrain_alt0_an_model_pt5_lr2_b29_new4.txt', 'D:/habr_minitest_alt0_an_model_pt5_lr2_b29_new4.txt', mytrain, mytest)
MiniCvScriptPasses('D:/habr_minitrain_alt0_an_model_pt5_lr3_b29_new4.txt', 'D:/habr_minitest_alt0_an_model_pt5_lr3_b29_new4.txt', mytrain, mytest)
MiniCvScriptPasses('D:/habr_minitrain_alt0_an_model_pt5_lr4_b29_new4.txt', 'D:/habr_minitest_alt0_an_model_pt5_lr4_b29_new4.txt', mytrain, mytest)
MiniCvScriptPasses('D:/habr_minitrain_alt0_an_model_pt5_lr5_b29_new4.txt', 'D:/habr_minitest_alt0_an_model_pt5_lr5_b29_new4.txt', mytrain, mytest)
MiniCvScriptPasses('D:/habr_minitrain_alt0_an_model_pt5_lr6_b29_new4.txt', 'D:/habr_minitest_alt0_an_model_pt5_lr6_b29_new4.txt', mytrain, mytest)
MiniCvScriptPasses('D:/habr_minitrain_alt0_an_model_pt5_lr7_b29_new4.txt', 'D:/habr_minitest_alt0_an_model_pt5_lr7_b29_new4.txt', mytrain, mytest)
MiniCvScriptPasses('D:/habr_minitrain_alt0_an_model_pt5_lr8_b29_new4.txt', 'D:/habr_minitest_alt0_an_model_pt5_lr8_b29_new4.txt', mytrain, mytest)
MiniCvScriptPasses('D:/habr_minitrain_alt0_an_model_pt5_lr9_b29_new4.txt', 'D:/habr_minitest_alt0_an_model_pt5_lr9_b29_new4.txt', mytrain, mytest)


84000
0.3662503875
0.994160848667
84000
0.438200809976
0.977683414528
84000
0.536111428929
0.955358395306
84000
0.526685098357
0.945323991194
84000
0.61482999256
0.935204435639
84000
0.609338695512
0.931345208083
84000
0.607578820607
0.92899057375
84000
0.601337942643
0.927870530167
84000
0.597379687512
0.927084901194


In [34]:
MiniCvScriptPasses('D:/habr_minitrain_alt0_an_model_pt1_lr5_b29_new4.txt', 'D:/habr_minitest_alt0_an_model_pt1_lr5_b29_new4.txt', mytrain, mytest)
MiniCvScriptPasses('D:/habr_minitrain_alt0_an_model_pt2_lr5_b29_new4.txt', 'D:/habr_minitest_alt0_an_model_pt2_lr5_b29_new4.txt', mytrain, mytest)
MiniCvScriptPasses('D:/habr_minitrain_alt0_an_model_pt3_lr5_b29_new4.txt', 'D:/habr_minitest_alt0_an_model_pt3_lr5_b29_new4.txt', mytrain, mytest)
MiniCvScriptPasses('D:/habr_minitrain_alt0_an_model_pt4_lr5_b29_new4.txt', 'D:/habr_minitest_alt0_an_model_pt4_lr5_b29_new4.txt', mytrain, mytest)
MiniCvScriptPasses('D:/habr_minitrain_alt0_an_model_pt5_lr5_b29_new4.txt', 'D:/habr_minitest_alt0_an_model_pt5_lr5_b29_new4.txt', mytrain, mytest)
MiniCvScriptPasses('D:/habr_minitrain_alt0_an_model_pt6_lr5_b29_new4.txt', 'D:/habr_minitest_alt0_an_model_pt6_lr5_b29_new4.txt', mytrain, mytest)
MiniCvScriptPasses('D:/habr_minitrain_alt0_an_model_pt7_lr5_b29_new4.txt', 'D:/habr_minitest_alt0_an_model_pt7_lr5_b29_new4.txt', mytrain, mytest)
MiniCvScriptPasses('D:/habr_minitrain_alt0_an_model_pt8_lr5_b29_new4.txt', 'D:/habr_minitest_alt0_an_model_pt8_lr5_b29_new4.txt', mytrain, mytest)
MiniCvScriptPasses('D:/habr_minitrain_alt0_an_model_pt9_lr5_b29_new4.txt', 'D:/habr_minitest_alt0_an_model_pt9_lr5_b29_new4.txt', mytrain, mytest)
len(mytrain), len(mytest)

84000
0.999831059988
0.925420878194
84000
0.939729814667
0.92350713175
84000
0.846570085512
0.924845981778
84000
0.717512880762
0.928988566972
84000
0.61482999256
0.935204435639
84000
0.441818052798
0.945541411083
84000
0.342750500976
0.954649288639
84000
0.310069121226
0.954331736833
84000
0.255405027702
0.943450090139


(84000, 36000)

In [35]:
MiniCvScriptPasses('D:/habr_minitrain_alt0_an_model_pt8_lr2_b29_new4.txt', 'D:/habr_minitest_alt0_an_model_pt8_lr2_b29_new4.txt', mytrain, mytest)


84000
0.328954191857
0.962572698806


In [46]:
myres = pd.read_csv('D:\\submission16.csv')

In [47]:
myres.describe()

,target
count,52913.000000
mean,2.519580
std,1.193069
min,0.000000
25%,1.663498
50%,2.553678
75%,3.352393
max,9.080687


In [55]:
myres2 = sample_sub.copy()
myres2.target = myres.target + 0.03

In [56]:
myres2.describe()

,target
count,0.0
mean,NaN
std,NaN
min,NaN
25%,NaN
50%,NaN
75%,NaN
max,NaN


In [57]:
sample_sub.head()

,target
url,
https://geektimes.ru/post/87455/,11.620054
https://geektimes.ru/post/87452/,4.822528
https://geektimes.ru/post/87459/,0.921104
https://habrahabr.ru/post/87461/,1.632126
https://habrahabr.ru/post/5754/,1.952122


In [58]:
myres2 = sample_sub

In [59]:
myres2.target.values = myres.target.values

AttributeError: can't set attribute

In [62]:
myres.describe()

,target
count,52913.000000
mean,2.519580
std,1.193069
min,0.000000
25%,1.663498
50%,2.553678
75%,3.352393
max,9.080687


In [63]:
your_submission15.describe()

,target
count,52913.000000
mean,2.519580
std,1.193069
min,0.000000
25%,1.663498
50%,2.553678
75%,3.352393
max,9.080687


In [81]:
myres2 = your_submission15.copy()
myres2.target += 0.3

In [82]:
myres2.describe()

,target
count,52913.000000
mean,2.819580
std,1.193069
min,0.300000
25%,1.963498
50%,2.853678
75%,3.652393
max,9.380687


In [91]:
myres2.describe(), your_submission15.describe()

(             target
 count  52913.000000
 mean       2.819580
 std        1.193069
 min        0.300000
 25%        1.963498
 50%        2.853678
 75%        3.652393
 max        9.380687,              target
 count  52913.000000
 mean       2.519580
 std        1.193069
 min        0.000000
 25%        1.663498
 50%        2.553678
 75%        3.352393
 max        9.080687)

In [90]:
your_submission23.describe()

,target
count,52913.000000
mean,2.622841
std,1.190220
min,0.000000
25%,1.790247
50%,2.643789
75%,3.422678
max,9.080687


In [84]:
myres2.to_csv('D:/submission34.csv')

In [156]:
newpred = MiniCvScriptPassesWar('D:/habr_mymodel_p92l18b29train_1.txt', 'D:/habr_mymodel_p92l18b29test_1.txt', train_target)

120000
0.188446434658


In [157]:
your_submission35 = sample_sub.copy()
your_submission35['target'] = newpred

In [159]:
your_submission35.describe()

,target
count,52913.000000
mean,2.663775
std,1.260969
min,0.000000
25%,1.754152
50%,2.687346
75%,3.574576
max,8.091794


In [160]:
newpred = MiniCvScriptPassesWar('D:/habr_mymodel_p85l3b29train_2.txt', 'D:/habr_mymodel_p85l3b29test_2.txt', train_target)

120000
0.187109807108


In [161]:
your_submission36 = sample_sub.copy()
your_submission36['target'] = newpred

In [162]:
your_submission36.describe()

,target
count,52913.000000
mean,2.653217
std,1.227022
min,0.000000
25%,1.782784
50%,2.656468
75%,3.571192
max,7.819062


In [164]:
your_submission35.describe()
your_submission35.to_csv('submission100.csv')

In [166]:
print(your_submission35.describe())
your_submission35.to_csv('D:/your_submissionlast.csv')

             target
count  52913.000000
mean       2.663775
std        1.260969
min        0.000000
25%        1.754152
50%        2.687346
75%        3.574576
max        8.091794


In [167]:
print(your_submission36.describe())
your_submission36.to_csv('D:/your_submissionlast2.csv')

             target
count  52913.000000
mean       2.653217
std        1.227022
min        0.000000
25%        1.782784
50%        2.656468
75%        3.571192
max        7.819062


In [179]:
newpred = MiniCvScriptPassesWar('D:/habr_mymodel_p83l35b29train_2.txt', 'D:/habr_mymodel_p83l35b29test_2.txt', train_target)


120000
0.213492474908


In [180]:
your_submission37 = sample_sub.copy()
your_submission37['target'] = newpred
your_submission37.describe()

,target
count,52913.000000
mean,2.620701
std,1.224971
min,0.000000
25%,1.748191
50%,2.622624
75%,3.540504
max,7.701097


In [181]:
your_submission37.to_csv('D:/your_submission239.csv')

myres2 = sample_sub